<a href="https://colab.research.google.com/github/rajy4683/EVAP2/blob/master/END_NoCoDeR_BPETokenize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sat Mar 13 05:32:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
import pandas as pd
import numpy as np
import fileinput
import re
import itertools
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
#import seaborn as sns
import dateutil.parser
import datetime
#from ipyfilechooser import FileChooser
import numpy as np
import os
import gzip
import dateutil.parser
from datetime import datetime
import sys
import glob
import matplotlib.dates as mdates
from datetime import timedelta
import ipywidgets as widgets
from IPython.display import display
import torch
import json
import random
import spacy
from pprint import pprint
import six
import sys, token, tokenize
import ast
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
import random
import math
import time
from torch.utils.data.distributed import DistributedSampler
#from tensorboardX import SummaryWriter
from tqdm import tqdm, trange
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
from torch.optim.lr_scheduler import ReduceLROnPlateau


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

     |████████████████████████████████| 1.9MB 8.1MB/s 
     |████████████████████████████████| 3.2MB 49.7MB/s 
     |████████████████████████████████| 890kB 54.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=3f6633259a077428075794322309d23cd828ca547eb20650c811066c97ba6d28
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler, WeightedRandomSampler
import tokenize

In [ ]:
%%bash
python -m spacy download en
python -m spacy download de

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp37-none-any.whl size=14907057 sha256=02f4971a5a268e1edd40de90dbf8e4c0695bd9946461c3e40f0d68b595720991
  Stored in directory: /tmp/pip-ephem-wheel-cache-menevs16/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [ ]:
!wget https://s3.amazonaws.com/code-search-net/CodeSearchNet/v2/python.zip
!wget -O gpt2_bpe_encoder.json https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/encoder.json

!unzip /content/python.zip
!cp /content/python/final/jsonl/train/python_train_0.jsonl.gz .
!gzip -d /content/python_train_0.jsonl.gz

!cp /content/drive/MyDrive/EVA4/END_Capstone/english_python_data.txt .
!cp /content/drive/MyDrive/EVA4/END_Capstone/english_python_cleaned.txt .
!cp /content/drive/MyDrive/EVA4/END_Capstone/end_capstone.csv .

--2021-03-13 05:34:57--  https://s3.amazonaws.com/code-search-net/CodeSearchNet/v2/python.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.114.117
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.114.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 940909997 (897M) [application/zip]
Saving to: ‘python.zip’

python.zip          100%[===================>] 897.32M  46.9MB/s    in 20s     

2021-03-13 05:35:17 (45.4 MB/s) - ‘python.zip’ saved [940909997/940909997]

--2021-03-13 05:35:17--  https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/encoder.json
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1042301 (1018K) [text/plain]
Saving to: ‘gpt2_bpe_encoder.json’

gpt2_bpe_encoder.js 100%[===================>]   1018K  --.-KB/s    in 0.05s  

In [ ]:
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel
from transformers import AutoTokenizer, AutoModel


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
auto_tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
spacy_en = spacy.load('en')
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]
#model = RobertaModel.from_pretrained("microsoft/codebert-base")
#model.to(device)

In [ ]:
"""
    Removes docstrings and comments
"""
def remove_docstrings_comments(src_string, doc_string=None, debug=False):
    mod = []

    prev_toktype = token.INDENT
    first_line = None
    last_lineno = -1
    last_col = 0
    try:
        #tokgen = tokenize.generate_tokens(source.readline)
        tokgen = tokenize.generate_tokens(six.StringIO(src_string.rstrip()).readline)
        for toktype, ttext, (slineno, scol), (elineno, ecol), ltext in tokgen:
            if 0:   # Change to if 1 to see the tokens fly by.
                print("%10s %-14s %-20r %r" % (
                    tokenize.tok_name.get(toktype, toktype),
                    "%d.%d-%d.%d" % (slineno, scol, elineno, ecol),
                    ttext, ltext
                    ))
            if slineno > last_lineno:
                last_col = 0
            if scol > last_col:
                mod.append(" " * (scol - last_col))
            if toktype == token.STRING and prev_toktype == token.INDENT:
                # Docstring
                mod.append("#--")
            elif toktype == tokenize.COMMENT:
                # Comment
                mod.append("##")
            else:
                mod.append(ttext)
            prev_toktype = toktype
            last_col = ecol
            last_lineno = elineno
        return "".join(mod)
    except:
        print(doc_string)
        print(src_string )
        print(sys.exc_info())

### CodeSearchNet

In [ ]:
columns_long_list = ['repo', 'path', 'url', 'code', 
                     'code_tokens', 'docstring', 'docstring_tokens', 
                     'language', 'partition']

with open("/content/python_train_0.jsonl",'r') as f:
    pd.concat([pd.read_json(f, 
                            orient='records', 
                            lines=True)[columns_long_list] 
                        ], sort=False)

In [ ]:
with open("/content/python_train_0.jsonl",'r') as f:
    my_df=pd.read_json(f, 
                orient='records', 
                lines=True)

In [ ]:
my_df_copy = my_df.loc[:20000,['docstring', 'code']].copy()

In [ ]:
my_df_copy[my_df_copy['cleaned_code_len'] <= 512].count()

docstring           11818
code                11818
docstring_len       11818
code_len            11818
cleaned_code        11818
cleaned_code_len    11818
dtype: int64

In [ ]:
my_df_copy['docstring_len'] =my_df_copy['docstring'].apply(lambda x: len(x))
my_df_copy['code_len'] = my_df_copy['code'].apply(lambda x: len(x))
my_df_copy['cleaned_code'] = my_df_copy.apply(lambda x: remove_docstrings_comments(x.code, x.docstring), axis=1)
my_df_copy['cleaned_code_len'] = my_df_copy['cleaned_code'].apply(lambda x: len(x))

In [ ]:
with open("gpt2_bpe_encoder.json") as f:
    gpt2_bpe=json.load(f)

In [ ]:
my_df_copy

In [ ]:
nl_to_pl_df = nl_to_pl_df.append(my_df_copy,ignore_index=True)

### Create datasets and dataloaders

In [ ]:
nl_to_pl_df = pd.read_csv('/content/end_capstone.csv')

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
print(nl_to_pl_df['code_len'].max(),nl_to_pl_df['code_len'].min())
print(nl_to_pl_df['docstring_len'].max(),nl_to_pl_df['docstring_len'].min())
nl_to_pl_df[nl_to_pl_df['code_len'] ==0]
nl_to_pl_df[(nl_to_pl_df['code_len'] > 256) & (nl_to_pl_df['code_len'] < 512)] 

2443 11
313 16


,docstring,code,docstring_len,code_len,cleaned_code,cleaned_code_len
5,# Write a program to check whether a number is...,num = 337\n\nif num > 1:\n for i in range(2...,60,311,num = 337\n\nif num > 1:\n for i in range(2...,308
7,# Write a program to find the factorial of a n...,num = 13\nfactorial = 1\n\nif num < 0:\n pr...,52,265,num = 13\nfactorial = 1\n\nif num < 0:\n pr...,262
53,# Write a function that takes in height(m) and...,"\ndef bmi(height: ""Meters"", weight: ""Kgs""):\n ...",98,450,"\ndef bmi(height: ""Meters"", weight: ""Kgs""):\n ...",447
62,# write a python program to sort dictionary it...,"dict1 = {'car': [7, 6, 3], \n 'bike': ...",50,262,"dict1 = {'car': [7, 6, 3], \n 'bike': ...",260
74,# write a python program to Get the maximum an...,"\nmy_dict = {'x':500, 'y':5874, 'z': 560}\n\nk...",78,276,"\nmy_dict = {'x':500, 'y':5874, 'z': 560}\n\nk...",274
...,...,...,...,...,...,...
4352,# Define a class named Shape and its subclass ...,\n\nclass Shape(object):\n def __init__(sel...,234,314,\n\nclass Shape(object):\n def __init__(sel...,312
4354,# Write a python program for a binary search f...,"import math\ndef bin_search(li, element):\n ...",171,390,"import math\ndef bin_search(li, element):\n ...",388
4359,# Write a python program to check if a number ...,"n = int(input(""Enter any number: ""))\nsum1 = 0...",67,261,"n = int(input(""Enter any number: ""))\nsum1 = 0...",259
4360,# Write a python program to Check if a Number ...,"sum1 = 0\nnum = int(input(""Enter a number:""))\...",65,347,"sum1 = 0\nnum = int(input(""Enter a number:""))\...",345


In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    # if n_gpu > 0:
    #     torch.cuda.manual_seed_all(seed)
set_seed(0x1112233)

In [ ]:
%%bash
git clone https://github.com/glample/fastBPE.git
git clone https://github.com/facebookresearch/TransCoder.git
pip install /content/fastBPE/.
g++ -std=c++11 -pthread -O3 /content/fastBPE/fastBPE/main.cc -IfastBPE -o fast
pip install sacrebleu=="1.2.11"


Processing ./fastBPE
  Created wheel for fastBPE: filename=fastBPE-0.1.1-cp37-cp37m-linux_x86_64.whl size=484395 sha256=ad26202b00ed7c05fed94f527589f4d823a1c1816a283c2cbbb01f06a13ada9a
  Stored in directory: /tmp/pip-ephem-wheel-cache-4u2iedmy/wheels/80/4e/ca/17a8ebf3e318c9540d60ef932be4247103012c4ae9d6c77237
Successfully built fastBPE
  Created wheel for sacrebleu: filename=sacrebleu-1.2.11-cp37-none-any.whl size=18641 sha256=ffe4a3232f6d22c632faf1fd54840e11f2007fa63ee22970bb8d8be3e6f0cdeb
  Stored in directory: /root/.cache/pip/wheels/93/0f/06/e1c5dcbca58e907c17b59be8e1f10ae4e43bb1fb68197a8d7c
  Created wheel for typing: filename=typing-3.7.4.3-cp37-none-any.whl size=26308 sha256=217735eedc0437f8b36420741927bd4b2f3ef793f58313108e5b32b678e88947
  Stored in directory: /root/.cache/pip/wheels/2d/04/41/8e1836e79581989c22eebac3f4e70aaac9af07b0908da173be
Successfully built sacrebleu typing


Cloning into 'fastBPE'...
Cloning into 'TransCoder'...


In [ ]:
!cp -rf /content/drive/MyDrive/EVA4/END_Capstone/cubert .
!pip install -r /content/cubert/requirements.txt
sys.path.append("/content/cubert/")

     |████████████████████████████████| 71kB 5.3MB/s 
     |████████████████████████████████| 92kB 6.5MB/s 
     |████████████████████████████████| 1.5MB 10.8MB/s 
     |████████████████████████████████| 412.3MB 41kB/s 
     |████████████████████████████████| 368kB 46.8MB/s 
     |████████████████████████████████| 368kB 48.2MB/s 
     |████████████████████████████████| 706kB 46.8MB/s 
     |████████████████████████████████| 194kB 55.3MB/s 
     |████████████████████████████████| 655kB 47.1MB/s 
     |████████████████████████████████| 983kB 44.8MB/s 
     |████████████████████████████████| 5.6MB 43.3MB/s 
     |████████████████████████████████| 81kB 11.7MB/s 
     |████████████████████████████████| 358kB 52.6MB/s 
     |████████████████████████████████| 3.8MB 38.4MB/s 
     |████████████████████████████████| 512kB 30.3MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 245kB 53.0MB/s 
     |████████████████████████████████| 163kB 45.5MB/s 

### Cubert Tokenizer

In [ ]:
"""A Python tokenizer subclass of CuBertTokenizer."""
import keyword
import re
import tokenize
import typing
from typing import Any
from typing import List
from typing import Sequence
from typing import Tuple
from absl import logging
from cubert import cubert_tokenizer
from cubert import unified_tokenizer


class PythonTokenizer2(cubert_tokenizer.CuBertTokenizer):
  """Tokenizer that extracts Python's lexical elements preserving strings."""
  _TOKEN_TYPE_MAP = {
      tokenize.COMMENT: unified_tokenizer.TokenKind.COMMENT,
      tokenize.DEDENT: unified_tokenizer.TokenKind.KEYWORD,
      tokenize.ENDMARKER: unified_tokenizer.TokenKind.EOS,
      tokenize.ERRORTOKEN: unified_tokenizer.TokenKind.ERROR,
      tokenize.INDENT: unified_tokenizer.TokenKind.KEYWORD,
      tokenize.NEWLINE: unified_tokenizer.TokenKind.NEWLINE,
      tokenize.NL: unified_tokenizer.TokenKind.PUNCTUATION,
      tokenize.NUMBER: unified_tokenizer.TokenKind.NUMBER,
      tokenize.OP: unified_tokenizer.TokenKind.PUNCTUATION,
      tokenize.STRING: unified_tokenizer.TokenKind.STRING,
  }
  _REVERSE_TOKEN_MAP = {
      cubert_tokenizer.token_from_token_type(tokenize.INDENT):
          tokenize.INDENT,
      cubert_tokenizer.token_from_token_type(tokenize.DEDENT):
          tokenize.DEDENT,
      unified_tokenizer.quote_special(unified_tokenizer.TokenKind.EOS.name):
          tokenize.ENDMARKER,
      unified_tokenizer.quote_special(unified_tokenizer.TokenKind.ERROR.name):
          tokenize.ERRORTOKEN,
      unified_tokenizer.quote_special(unified_tokenizer.TokenKind.NEWLINE.name):
          tokenize.NEWLINE,
      cubert_tokenizer.token_from_token_type(tokenize.NL):
          tokenize.NL,
  }
  # Adding the end-of-string anchor \Z below, since re.fullmatch wasn't
  # available in Python2.
  _NUMBERS = re.compile('(' + tokenize.Number + r')\Z')  # pytype: disable=module-attr
  _SINGLE_STRINGS = re.compile('(' + tokenize.String + r')\Z')  # pytype: disable=module-attr
  _TRIPLE_STRING_BEGINNINGS = re.compile(tokenize.Triple)  # pytype: disable=module-attr
  _COMMENTS = re.compile('(' + tokenize.Comment + r')\Z')  # pytype: disable=module-attr

  _EXACT_TOKEN_TYPES = tokenize.EXACT_TOKEN_TYPES.keys()  # pytype: disable=module-attr

  # Token types that CubertTokenizer will tokenize by their type and not
  # content.
  _TOKEN_TYPES_TO_TOKENIZE_BY_TYPE = [
      tokenize.NEWLINE, tokenize.DEDENT, tokenize.NL
  ]

  def tokenize_and_abstract(
      self,
      source_code):
    """Produces a language-agnostic tokenization of the input code."""
    agnostic_tokens: List[unified_tokenizer.AbstractToken] = []

    try:
      token_tuples = unified_tokenizer.code_to_tokens(source_code)
    except (tokenize.TokenError, IndentationError) as e:
      logging.warning('The tokenizer raised exception `%s` while parsing %s', e,
                      source_code)

      # We don't try to do recovery from errors quite yet. Emit just an
      # error and end-of-sequence and return.
      agnostic_tokens.append(
          unified_tokenizer.AbstractToken(
              unified_tokenizer.quote_special(
                  unified_tokenizer.TokenKind.ERROR.name),
              unified_tokenizer.TokenKind.ERROR,
              unified_tokenizer.TokenMetadata(
                  start=unified_tokenizer.Position(
                      line=0, column=0),
                  end=unified_tokenizer.Position(
                      line=0, column=0))))
      agnostic_tokens.append(
          unified_tokenizer.AbstractToken(
              unified_tokenizer.quote_special(
                  unified_tokenizer.TokenKind.EOS.name),
              unified_tokenizer.TokenKind.EOS,
              unified_tokenizer.TokenMetadata(
                  start=unified_tokenizer.Position(
                      line=0, column=0),
                  end=unified_tokenizer.Position(
                      line=0, column=0))))
      return agnostic_tokens

    for token_tuple in token_tuples:
      spelling = token_tuple.string
      kind = token_tuple.type

      # We'll adjust the spelling of some tokens, e.g., those that we
      # tokenize by their type rather than their original spelling. Indentation
      # and dedentation tokens are like that.
      adjusted_spelling = spelling
      token_kind = unified_tokenizer.TokenKind.NONE
      if kind == tokenize.NAME:
        # Disambiguate identifiers from keywords.
        if keyword.iskeyword(spelling):
          token_kind = unified_tokenizer.TokenKind.KEYWORD
        else:
          token_kind = unified_tokenizer.TokenKind.IDENTIFIER
      else:
        if kind in PythonTokenizer2._TOKEN_TYPES_TO_TOKENIZE_BY_TYPE:
          # Replace spelling with type.
          adjusted_spelling = cubert_tokenizer.token_from_token_type(kind)
        elif kind is tokenize.INDENT:
          # For INDENT, in particular, we also record the actual spelling too.
          adjusted_spelling = '{indent}{spelling}'.format(
              indent=cubert_tokenizer.token_from_token_type(kind),
              spelling=spelling)
          #print(adjusted_spelling)
        elif kind == tokenize.ENDMARKER:
          adjusted_spelling = unified_tokenizer.quote_special(
              unified_tokenizer.TokenKind.EOS.name)

        # Map everything according to table.
        try:
          token_kind = PythonTokenizer2._TOKEN_TYPE_MAP[kind]
        except KeyError as ke:
          # It's possible we're here because of async/await. Those kept being
          # turned into keywords and then removed from keywords, so we can't
          # rely on knowing which they are. We'll check by spelling.
          # See: https://bugs.python.org/issue30406
          # and https://bugs.python.org/issue33260
          # and https://bugs.python.org/issue35975
          if spelling in ('async', 'await'):
            token_kind = unified_tokenizer.TokenKind.KEYWORD
          else:
            raise ValueError('While trying to turn Python token %r into an '
                             'agnostic one, raised %r.' %
                             ((spelling, kind), ke))

      start_line, start_column = token_tuple.start
      end_line, end_column = token_tuple.end
      # Unlike other languages, NEWLINE tokens are reported as ending on the
      # same line as where they started. We adjust that here, to stick to the
      # same convention as other tokenizers.
      if ((token_kind == unified_tokenizer.TokenKind.NEWLINE) or
          (kind == tokenize.NL)):
        end_line = start_line + 1
        end_column = 0

      agnostic_tokens.append(
          unified_tokenizer.AbstractToken(
              spelling=adjusted_spelling, kind=token_kind,
              metadata=unified_tokenizer.TokenMetadata(
                  # Python's tokenizer counts lines starting from 1, so we
                  # have to offset what we read from the `TokenInfo` tuple.
                  start=unified_tokenizer.Position(
                      line=start_line - 1, column=start_column),
                  end=unified_tokenizer.Position(
                      line=end_line - 1, column=end_column))))
    #print(agnostic_tokens)
    return agnostic_tokens

  def untokenize_abstract(self, whole_tokens):
    # Reconstruct Python tokenizer tuples, so that Python's untokenize can be
    # invoked.
    token_tuples: List[Tuple[int, str]] = []

    for whole_token in whole_tokens:
      if whole_token in PythonTokenizer2._EXACT_TOKEN_TYPES:
        token_tuples.append((tokenize.OP, whole_token))
      elif cubert_tokenizer.token_from_token_type(
          tokenize.INDENT) in whole_token:
        # We baked the type and spelling into one token. Break them up.
        spelling = whole_token.replace(
            cubert_tokenizer.token_from_token_type(tokenize.INDENT), '')
        token_tuples.append((tokenize.INDENT, spelling))
      elif whole_token in PythonTokenizer2._REVERSE_TOKEN_MAP:
        python_kind = PythonTokenizer2._REVERSE_TOKEN_MAP[whole_token]
        if python_kind in (tokenize.DEDENT, tokenize.ENDMARKER,
                           tokenize.ERRORTOKEN):
          spelling = ''
        else:  # python_kind in (tokenize.NEWLINE, tokenize.NL)
          spelling = '\n'
        token_tuples.append((python_kind, spelling))
      elif keyword.iskeyword(whole_token):
        token_tuples.append((tokenize.NAME, whole_token))
      elif PythonTokenizer2._NUMBERS.match(whole_token):
        token_tuples.append((tokenize.NUMBER, whole_token))
      elif PythonTokenizer2._SINGLE_STRINGS.match(whole_token):
        token_tuples.append((tokenize.STRING, whole_token))
      elif PythonTokenizer2._TRIPLE_STRING_BEGINNINGS.match(whole_token):
        token_tuples.append((tokenize.STRING, whole_token))
      elif PythonTokenizer2._COMMENTS.match(whole_token):
        token_tuples.append((tokenize.COMMENT, whole_token))
      else:
        # Everything else we map back to NAME.
        token_tuples.append((tokenize.NAME, whole_token))

    reconstructed = tokenize.untokenize(typing.cast(Any, token_tuples))
    return reconstructed



In [ ]:
def get_lang_specific_tokens(init_tokenizer, code_snip):
    #tokens_complete = init_tokenizer.tokenize(source_code=code_snip)
    tokens = init_tokenizer.tokenize_and_abstract(source_code=code_snip )
    conditioned = init_tokenizer.condition_full_tokens(tokens)
    agnostic_token_lists = unified_tokenizer._agnostic_tokens_to_lists_of_token_lists(conditioned)
    with_identifiers_heuristically_split = unified_tokenizer._subtokenize_identifiers_heuristically(
        agnostic_token_lists)
    with_string_tokens_heuristically_split = unified_tokenizer._subtokenize_strings_heuristically(
        with_identifiers_heuristically_split)
    shortened_subtokens = unified_tokenizer._shorten_subtokens(with_string_tokens_heuristically_split, 20)
    sanitization_mapping = init_tokenizer.get_mappings()
    subtoken_lists = unified_tokenizer.sanitize_subtoken_lists(shortened_subtokens,
                                            sanitization_mapping,
                                            unified_tokenizer.SENTINEL)
    #flat_toks =unified_tokenizer.flatten_subtoken_lists(subtoken_lists)
    test_spellings = []
    test_tok_types = []
    for t in subtoken_lists:
        #if(len(t.spelling) == 1):
        #print(len(t.spellings))
        test_spellings.extend(t.spellings)
        match=False
        for cubert_token in set(init_tokenizer._REVERSE_TOKEN_MAP.keys()):
            #print("Checking for:",cubert_token)
            if cubert_token in t.spellings[0]:
                #print(t.spellings)
                selected_token = tokenize.tok_name[init_tokenizer._REVERSE_TOKEN_MAP[cubert_token]]
                test_tok_types.extend([selected_token]*len(t.spellings))
                match=True          
        if match == False:
            test_tok_types.extend([t.kind.name]*len(t.spellings))
    return test_spellings, test_tok_types



### Cubert Vectorizer

In [ ]:
class NewVectorizer():
    def __init__(self, code_piece, tok_type_counter):
        self.code_piece = code_piece
        self.tok_type_counter = tok_type_counter
        self.code_word2idx = {'<s>':0,'</s>':1,'<pad>':2, '<unk>':3}
        self.code_idx2word = {v:k for k,v in self.code_word2idx.items()}
        self.toktype_word2idx = {'<s>':0,'</s>':1,'<pad>':2, '<unk>':3}
        self.toktype_idx2word = {v:k for k,v in self.toktype_word2idx.items()}
        self.max_tok_length = len(self.toktype_word2idx)
        self.max_code_length = len(self.code_word2idx)
        self.UNK_FOR_TOKEN_TYPE = '<unk>'
        self.UNK_FOR_CODEPIECE = '<unk>'
        self.ID_UNK_TOKEN_TYPE = self.toktype_word2idx[self.UNK_FOR_TOKEN_TYPE]
        self.ID_UNK_CODEPIECE = self.code_word2idx[self.UNK_FOR_CODEPIECE]

        self.PAD_FOR_CODEPIECE = '<pad>'
        self.PAD_FOR_TOKEN_TYPE = '<pad>'
        self.ID_PAD_FOR_CODEPIECE = self.code_word2idx['<pad>']
        self.ID_PAD_FOR_TOKEN_TYPE = self.toktype_word2idx['<pad>']
        
        self.SOS_FOR_CODEPIECE = '<s>'
        self.SOS_FOR_TOKEN_TYPE = '<s>'
        self.ID_SOS_FOR_CODEPIECE = self.code_word2idx['<s>']
        self.ID_SOS_FOR_TOKEN_TYPE = self.toktype_word2idx['<s>']

        self.EOS_FOR_CODEPIECE = '</s>'
        self.EOS_FOR_TOKEN_TYPE = '</s>'
        self.ID_EOS_FOR_CODEPIECE = self.code_word2idx['</s>']
        self.ID_EOS_FOR_TOKEN_TYPE = self.toktype_word2idx['</s>']
        self.build_vocab()

    def build_vocab(self):
        idx=len(self.code_word2idx.keys())
        for k in self.code_piece.keys():
            self.code_word2idx[k]=idx
            self.code_idx2word[idx]=k
            idx += 1
        
        idx=len(self.toktype_word2idx.keys())
        for k in self.tok_type_counter.keys():
            self.toktype_word2idx[k]=idx
            self.toktype_idx2word[idx]=k
            idx += 1

        self.max_tok_length = len(self.toktype_word2idx.keys())
        self.max_code_length = len(self.code_word2idx.keys())
    ### Returns the code piece for a given ID
    def convert_id_to_codepiece(self, id_for_code):
        if(id_for_code not in list(self.code_idx2word.keys())):
            return self.UNK_FOR_CODEPIECE
        return self.code_idx2word[id_for_code]
    ### Returns the ID for a given code piece
    def convert_codepiece_to_id(self, code_piece):
        if(code_piece not in list(self.code_word2idx.keys())):
            return self.ID_UNK_CODEPIECE
        return self.code_word2idx[code_piece]
    ### Returns the TOKEN ID for a given TOKEN type
    def convert_toktype_to_id(self, tok_piece):
        if(tok_piece not in list(self.toktype_word2idx.keys())):
            print("No match for",tok_piece)
            return self.ID_UNK_TOKEN_TYPE
        return self.toktype_word2idx[tok_piece]
    ### Returns the TOKEN type for a given TOKEN ID
    def convert_id_to_toktype(self, id_for_toktype):
        if(id_for_toktype not in list(self.toktype_idx2word.keys())):
            return self.UNK_FOR_TOKEN_TYPE
        return self.toktype_idx2word[id_for_toktype]

In [ ]:
import cubert
from absl import app
from absl import flags
from tensor2tensor.data_generators import text_encoder
import enum
import cubert_tokenizer
from cubert import code_to_subtokenized_sentences
#from cubert import tokenizer_registry
from cubert import python_tokenizer
import python_tokenizer
from tensor2tensor.data_generators import text_encoder_build_subword
from collections import Counter, defaultdict

@enum.unique
class TokenizerEnum(enum.Enum):
  """Enum for Tokenizers."""
  #PYTHON = python_tokenizer.PythonTokenizer
  PYTHON = PythonTokenizer2

In [ ]:
#### Generate and create the Vectorizer instance
word_counter=Counter()
init_tokenizer=PythonTokenizer2()
tok_type_counter = Counter()
for code_snip in nl_to_pl_df['cleaned_code']:
    toks, tok_types = get_lang_specific_tokens(init_tokenizer, code_snip)
    word_counter.update(Counter(toks))
    tok_type_counter.update(Counter(tok_types))

In [ ]:
code_tok_vectorizer = NewVectorizer(word_counter, tok_type_counter)

In [ ]:
print(code_tok_vectorizer.max_code_length,code_tok_vectorizer.max_tok_length)

5813 16


In [ ]:
[ code_tok_vectorizer.convert_id_to_toktype(toktype) for toktype in tok_type_ids] == tok_types

True

### Old NLP DS

In [ ]:
tok_ids_list=[]
class NLPLSingleEntry(object):
    """A single set of features of data."""

    def __init__(self, 
                 code_ids,
                 tok_ids,
                 code_mask, 
                 doc_ids,
                 doc_mask,
                 ):
        self.code_ids = code_ids
        self.code_mask = code_mask
        self.tok_ids = tok_ids
        self.doc_ids = doc_ids
        self.doc_mask = doc_mask
        #self.segment_ids = segment_ids
        #self.label_id = label_id
class NLPLDataSet():
    def __init__(self, 
                 doc_tokenizer, 
                 code_tokenizer,
                 code_tok_vectorizer):
        self.doc_tokenizer = doc_tokenizer
        self.code_tokenizer = code_tokenizer
        self.code_tok_vectorizer = code_tok_vectorizer

    def prepare_tokens(self, 
                       samples, 
                       tokenizer, 
                       max_seq_length=0,
                       data_type=None):
        """
            Tokenizes an input sequence, adds padding and SOS+EOS 
        """
        toks = tokenizer.tokenize(samples.lower())
        # print(data_type)
        # print(toks)
        if max_seq_length > 2 and len(toks) > max_seq_length - 2:
            toks = toks[:max_seq_length -2]
        tok_ids =  tokenizer.convert_tokens_to_ids(toks)
        ### We use pseudo-BERT process so we will add both CLS and SEP tokens for
        ### src and target inputs
        tok_ids = [tokenizer.cls_token_id] + tok_ids + [tokenizer.sep_token_id]
        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [ 1 ] * len(tok_ids)

        if len(tok_ids) < max_seq_length:
            padding_length = max_seq_length - len(tok_ids)
            tok_ids = tok_ids + ([tokenizer.pad_token_id] * padding_length)
            input_mask = input_mask + ([ 0 ] * padding_length) ### Padded tokens are zero-masked
        
        # print(tok_ids)
        return tok_ids, input_mask
    def prepare_code_tokens(self, 
                       samples, 
                       tokenizer, 
                       max_seq_length=0,
                       data_type=None):
        """
            Tokenizes an input sequence, adds padding and SOS+EOS 
        """
        _toks, _tok_types = get_lang_specific_tokens(self.code_tokenizer, samples)
        #print(_tok_types)
        # print(data_type)
        # print(toks)
        if max_seq_length > 2 and len(_toks) > max_seq_length - 2:
            _toks = _toks[:max_seq_length -2]
            _tok_types = _tok_types[:max_seq_length -2]
        #tok_ids =  tokenizer.convert_tokens_to_ids(toks)
        tok_ids = [ self.code_tok_vectorizer.convert_codepiece_to_id(code) for code in _toks]
        tok_types = [ self.code_tok_vectorizer.convert_toktype_to_id(toktype) for toktype in _tok_types]
        
        ### We use pseudo-BERT process so we will add both CLS and SEP tokens for
        ### src and target inputs
        tok_ids = [self.code_tok_vectorizer.ID_SOS_FOR_CODEPIECE] + tok_ids + [self.code_tok_vectorizer.ID_EOS_FOR_CODEPIECE]
        tok_types = [self.code_tok_vectorizer.ID_SOS_FOR_TOKEN_TYPE] + tok_types + [self.code_tok_vectorizer.ID_EOS_FOR_TOKEN_TYPE]
        #print(len(tok_ids), len(tok_types))
        assert(len(tok_ids) == len(tok_types))
        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [ 1 ] * len(tok_ids)

        if len(tok_ids) < max_seq_length:
            padding_length = max_seq_length - len(tok_ids)
            tok_ids = tok_ids + ([self.code_tok_vectorizer.ID_PAD_FOR_CODEPIECE] * padding_length)
            tok_types = tok_types + ([self.code_tok_vectorizer.ID_PAD_FOR_TOKEN_TYPE] * padding_length)
            input_mask = input_mask + ([ 0 ] * padding_length) ### Padded tokens are zero-masked
        
        # print(tok_ids)
        return tok_ids, tok_types, input_mask

    def create_dataset(self,
                    nl_to_pl_df,
                    final_ds,
                    sample_count=10000,
                    max_doc_len=50,
                    max_code_len=0):
        """
            Reads from a dataframe, tokenizes and numericalizes both docstrings 
            and code. 

        """
        #final_ds = []
        for idx in nl_to_pl_df.itertuples():
            ## For SOS and EOS tokens 2 positions are left
            if not idx.cleaned_code:
                print("Invalid entry, No code found for:", idx.docstring)
            
            doc_toks, doc_mask = self.prepare_tokens(idx.docstring,
                                                      self.doc_tokenizer,
                                                      max_doc_len,
                                                      "docs")
            code_ids, tok_ids, code_mask = self.prepare_code_tokens(idx.cleaned_code,
                                                      self.code_tokenizer,
                                                      max_code_len,
                                                      "code")
            #code_toks = None
            ### Skip over current iteration if no valid code found

            # print(code_toks)
            # print(code_mask)
            # print(doc_toks)
            # print(doc_mask)
            final_entry = NLPLSingleEntry(code_ids,
                                          tok_ids,
                                          code_mask, 
                                          doc_toks, 
                                          doc_mask)
            # print(final_entry.code_ids)
            # print(final_entry.code_mask)
            # print(final_entry.doc_ids)
            # print(final_entry.doc_mask)
            final_ds.append(final_entry)
        #print(len(final_ds))
        return final_ds
        

### TransCoder Tokenizer + FastBPE + TransCoder DataSet Creator

In [ ]:
!pip install clang

In [ ]:
sys.path.append("/content/TransCoder")
import clang
from clang.cindex import TokenKind

from preprocessing.src.timeout import timeout, TimeoutError
import preprocessing.src.javalang_tokenizer as javalang_tok
#from preprocessing.src.code_tokenizer import tokenize_python, detokenize_python
clang.cindex.Config.set_library_path('/usr/lib/llvm-6/lib/')
clang.cindex.Config.set_library_file("/usr/lib/llvm-6.0/lib/libclang-6.0.so.1")

In [ ]:
# sys.path.append("/content/TransCoder")
# import clang
# from clang.cindex import TokenKind

# from preprocessing.src.timeout import timeout, TimeoutError
# import preprocessing.src.javalang_tokenizer as javalang_tok
# #from preprocessing.src.code_tokenizer import tokenize_python, detokenize_python
# clang.cindex.Config.set_library_path('/usr/lib/llvm-6/lib/')
# clang.cindex.Config.set_library_file("/usr/lib/llvm-6.0/lib/libclang-6.0.so.1")

import argparse
import logging
import os
import random
import re
import sys
import tokenize
from io import BytesIO

import preprocessing.src.javalang_tokenizer as javalang_tok
from clang.cindex import TokenKind
from preprocessing.src.timeout import timeout, TimeoutError
from sacrebleu import tokenize_v14_international

TOK_NO_SPACE_BEFORE = {',', ';'}
#clang.cindex.Config.set_library_path('/usr/lib/llvm-7/lib/')
STRINGS_AND_COMMENTS_TOKEN_KINDS = {TokenKind.LITERAL, TokenKind.COMMENT}
logging.basicConfig(
    filename='timeout_cpp_tokenizer_examples.log', level=logging.DEBUG)

idx = clang.cindex.Index.create()

JAVA_TOKEN2CHAR = {'STOKEN0': "//",
                   'STOKEN1': "/*",
                   'STOKEN2': "*/",
                   'STOKEN3': "/**",
                   'STOKEN4': "**/",
                   'STOKEN5': '"""',
                   'STOKEN6': '\\n'
                   }
JAVA_CHAR2TOKEN = {"//": ' STOKEN0 ',
                   "/*": ' STOKEN1 ',
                   "*/": ' STOKEN2 ',
                   "/**": ' STOKEN3 ',
                   "**/": ' STOKEN4 ',
                   '"""': ' STOKEN5 ',
                   '\\n': ' STOKEN6 '
                   }

CPP_TOKEN2CHAR = JAVA_TOKEN2CHAR.copy()
CPP_CHAR2TOKEN = JAVA_CHAR2TOKEN.copy()

PYTHON_TOKEN2CHAR = {'STOKEN0': '#',
                     'STOKEN1': "\\n",
                     'STOKEN2': '"""',
                     'STOKEN3': "'''"
                     }

PYTHON_CHAR2TOKEN = {'#': ' STOKEN0 ',
                     "\\n": ' STOKEN1 ',
                     '"""': ' STOKEN2 ',
                     "'''": ' STOKEN3 '
                     }


class ind_iter(object):
    def __init__(self, len):
        self.i = 0
        self.len = len

    def next(self):
        self.i += 1
        if self.i > (self.len - 1):
            raise StopIteration

    def prev(self):
        self.i -= 1
        if self.i < 0:
            raise StopIteration


def process_string(tok, char2tok, tok2char, is_comment):
    if is_comment:
        tok = re.sub(' +', ' ', tok)
        tok = re.sub(r"(.)\1\1\1\1+", r"\1\1\1\1\1", tok)
        if len(re.sub(r'\W', '', tok)) < 2:
            return ''
    tok = tok.replace(' ', ' ▁ ')
    for char, special_token in char2tok.items():
        tok = tok.replace(char, special_token)
    if tok.startswith(' STOKEN0'):
        if tok.endswith('\n'):
            tok = tok[:-1]
        tok += ' ENDCOM'
    tok = tok.replace('\n', ' STRNEWLINE ')
    tok = tok.replace('\t', ' TABSYMBOL ')
    tok = re.sub(' +', ' ', tok)
    tok = tokenize_v14_international(tok)
    tok = re.sub(' +', ' ', tok)
    for special_token, char in tok2char.items():
        tok = tok.replace(special_token, char)
    tok = tok.replace('\r', '')

    return tok


def tokenize_python(s, keep_comments=False):
    try:
        assert isinstance(s, str)
        s = s.replace(r'\r', '')
        tokens = []

        try:
            iterator = tokenize.tokenize(BytesIO(s.encode('utf-8')).readline)
        except SyntaxError as excep:
            raise SyntaxError(excep)

        removed_docstr = 0
        while True:
            try:
                toktype, tok, _, _, line = next(iterator)
            except (tokenize.TokenError, IndentationError, SyntaxError, UnicodeDecodeError):
                raise Exception(
                    f"Impossible to parse tokens because icorrect source code \"{s[0:30]}\" ...")
            except StopIteration:
                raise Exception(f"End of iterator before ENDMARKER token.")

            if toktype == tokenize.ENCODING or toktype == tokenize.NL:
                continue

            elif toktype == tokenize.NEWLINE:
                if removed_docstr == 1:
                    removed_docstr = 0
                    continue
                tokens.append('NEW_LINE')

            elif toktype == tokenize.COMMENT:
                if keep_comments:
                    com = process_string(
                        tok, PYTHON_CHAR2TOKEN, PYTHON_TOKEN2CHAR, True)
                    if len(com) > 0:
                        tokens.append(com)
                else:
                    continue

            elif toktype == tokenize.STRING:
                if tok == line.strip():  # docstring
                    if not keep_comments:
                        removed_docstr = 1
                        continue
                    else:
                        coms = process_string(
                            tok, PYTHON_CHAR2TOKEN, PYTHON_TOKEN2CHAR, True)
                        if len(coms) > 0:
                            tokens.append(coms)
                        else:
                            removed_docstr = 1
                else:
                    tokens.append(process_string(
                        tok, PYTHON_CHAR2TOKEN, PYTHON_TOKEN2CHAR, False))

            elif toktype == tokenize.INDENT:
                tokens.append('INDENT')

            elif toktype == tokenize.DEDENT:
                # empty block
                if tokens[-1] == 'INDENT':
                    tokens = tokens[:-1]
                else:
                    tokens.append('DEDENT')

            elif toktype == tokenize.ENDMARKER:
                tokens.append('ENDMARKER')
                break

            else:
                tokens.append(tok)

        assert (tokens[-1] == 'ENDMARKER'), "Error, no end marker"
        return tokens[:-1]
    except KeyboardInterrupt:
        raise
    except:
        return []


def detokenize_python(s):
    try:
        assert isinstance(s, str) or isinstance(s, list)
        if isinstance(s, list):
            s = ' '.join(s)
        s = s.replace('ENDCOM', 'NEW_LINE')
        s = s.replace('▁', 'SPACETOKEN')

        lines = s.split('NEW_LINE')
        tabs = ''
        for i, line in enumerate(lines):
            line = line.strip()
            if line.startswith('INDENT '):
                tabs += '    '
                line = line.replace('INDENT ', tabs)
            elif line.startswith('DEDENT'):
                number_dedent = line.count('DEDENT')
                tabs = tabs[4 * number_dedent:]
                line = line.replace("DEDENT", '')
                line = line.strip()
                line = tabs + line
            elif line == 'DEDENT':
                line = ''
            else:
                line = tabs + line
            lines[i] = line
        untok_s = '\n'.join(lines)

        # find string and comment with parser and detokenize string correctly
        try:
            for toktype, tok, _, _, line in tokenize.tokenize(BytesIO(untok_s.encode('utf-8')).readline):
                if toktype == tokenize.STRING or toktype == tokenize.COMMENT:
                    tok_ = tok.replace('STRNEWLINE', '\n').replace(
                        'TABSYMBOL', '\t').replace(' ', '').replace('SPACETOKEN', ' ')
                    untok_s = untok_s.replace(tok, tok_)
        except KeyboardInterrupt:
            raise
        except:
            pass

        # detokenize imports
        untok_s = untok_s.replace('. ', '.').replace(' .', '.').replace(
            'import.', 'import .').replace('from.', 'from .')
        untok_s = untok_s.replace('> >', '>>').replace('< <', '<<')
        return untok_s
    except KeyboardInterrupt:
        raise
    except:
        return ''


def extract_functions_python_with_docstring(function):
    ds = re.findall(
        "[:][ ][N][E][W][_][L][I][N][E][ ][I][N][D][E][N][T][ ]['][']['].*?['][']['][ ][N][E][W][_][L][I][N][E]|[:][ ][N][E][W][_][L][I][N][E][ ][I][N][D][E][N][T][ ][\"][\"][\"].*?[\"][\"][\"][ ][N][E][W][_][L][I][N][E]",
        function, re.DOTALL)
    if len(ds) > 0:
        for d in ds:
            function = function.replace(d[18:-9], '')
        coms = ' '.join([d[18:-9] for d in ds])
        inline_coms = re.findall('[#].*?[E][N][D][C][O][M]', function)
        for inline_com in inline_coms:
            function = function.replace(inline_com, '')
            coms += ' <INLINE> '
            coms += inline_com
        if len(re.sub(r'\W', '', coms.replace('<INLINE>', '').replace('ENDCOM', ''))) < 5:
            return '', ''
        else:
            return re.sub('\s+', ' ', function), coms
    else:
        return '', ''


def extract_functions_python(s):
    tokens = iter(s.split())
    functions_standalone = []
    functions_class = []
    number_indent = 0
    try:
        token = next(tokens)
    except StopIteration:
        return [], []
    while True:
        try:
            if token == 'def':
                function = ['def']
                while not (token == 'DEDENT' and number_indent == 0):
                    token = next(tokens)
                    if token == 'INDENT':
                        number_indent += 1
                    elif token == 'DEDENT':
                        number_indent -= 1
                    function.append(token)
                try:
                    if function[function.index('(') + 1] == 'self':
                        function = filter_functions_python_2_3(
                            ' '.join(function))
                        if function is not None:
                            functions_class.append(function)
                    else:
                        function = filter_functions_python_2_3(
                            ' '.join(function))
                        if function is not None:
                            functions_standalone.append(function)
                except:
                    print(function)
                    token = next(tokens)
            else:
                token = next(tokens)
        except StopIteration:
            break
    return functions_standalone, functions_class


def filter_functions_python_2_3(function):
    if (re.search("print [^(]", function) is None and
            re.search("raise \w+ ,", function) is None and
            re.search("except \w+ ,", function) is None and
            re.search("[^ ]+ = \d+ L", function) is None and
            ". iterkeys ( )" not in function and
            ". itervalues ( )" not in function and
            ". iteritems ( )" not in function and
            "xrange (" not in function and
            "imap (" not in function):
        return function
    else:
        return None


def get_function_name_python(s):
    assert isinstance(s, str) or isinstance(s, list)
    if isinstance(s, str):
        s = s.split()
    return s[s.index('def') + 1]


@timeout(10)
def get_cpp_tokens_and_types(s):
    tokens = []
    assert isinstance(s, str)
    s = s.replace(r'\r', '')
    hash = str(random.getrandbits(128))
    parsed_code = idx.parse(
        hash + '_tmp.cpp', args=['-std=c++11'], unsaved_files=[(hash + '_tmp.cpp', s)], options=0)
    for tok in parsed_code.get_tokens(extent=parsed_code.cursor.extent):
        tokens.append((tok.spelling, tok.kind))
    return tokens


def tokenize_cpp(s, keep_comments=False):
    tokens = []
    assert isinstance(s, str)
    try:
        tokens_and_types = get_cpp_tokens_and_types(s)
        for tok, typ in tokens_and_types:
            if not keep_comments and typ == TokenKind.COMMENT:
                continue
            if typ in STRINGS_AND_COMMENTS_TOKEN_KINDS:
                if typ == TokenKind.COMMENT:
                    com = process_string(
                        tok, CPP_CHAR2TOKEN, CPP_TOKEN2CHAR, True)
                    if len(com) > 0:
                        tokens.append(com)
                else:
                    tokens.append(process_string(
                        tok, CPP_CHAR2TOKEN, CPP_TOKEN2CHAR, False))
            else:
                tokens.append(tok)
        return tokens
    except KeyboardInterrupt:
        raise
    except TimeoutError:
        print(f'TimeOut Error')
        logging.info('*' * 20)
        logging.info(f'TimeOut Error for string {s}')
        return []
    except:
        return []


def tokenize_java(s, keep_comments=False):
    try:
        tokens = []
        assert isinstance(s, str)
        s = s.replace(r'\r', '')
        tokens_generator = javalang_tok.tokenize(
            s, keep_comments=keep_comments)
        for token in tokens_generator:
            if isinstance(token, javalang_tok.String):
                tokens.append(process_string(
                    token.value, JAVA_CHAR2TOKEN, JAVA_TOKEN2CHAR, False))
            elif isinstance(token, javalang_tok.Comment):
                com = process_string(
                    token.value, JAVA_CHAR2TOKEN, JAVA_TOKEN2CHAR, True)
                if len(com) > 0:
                    tokens.append(com)
            else:
                tokens.append(token.value)
        return tokens
    except:
        return []


def detokenize_cpp(s):
    assert isinstance(s, str) or isinstance(s, list)
    if isinstance(s, list):
        s = ' '.join(s)
    # the ▁ character created bugs in the cpp tokenizer
    s = s.replace('ENDCOM', '\n').replace('▁', ' SPACETOKEN ')
    try:
        tokens_and_types = get_cpp_tokens_and_types(s)
    except:
        return ''
    new_tokens = []
    i = 0
    while i < len(tokens_and_types):
        token, type = tokens_and_types[i]
        if type in STRINGS_AND_COMMENTS_TOKEN_KINDS:
            new_tokens.append(token.replace('STRNEWLINE', '\n').replace(
                'TABSYMBOL', '\t').replace(' ', '').replace('SPACETOKEN', ' '))
            if type == TokenKind.COMMENT:
                new_tokens.append('NEW_LINE')
        elif token == '}':
            if i < len(tokens_and_types) - 1 and tokens_and_types[i + 1][0] == ';':
                new_tokens += ['CB_COLON', 'NEW_LINE']
                i += 2
                continue
            if i < len(tokens_and_types) - 1 and tokens_and_types[i + 1][0] == ',':
                new_tokens += ['CB_COMA', 'NEW_LINE']
                i += 2
                continue
            new_tokens += ['CB_', 'NEW_LINE']
        elif token == '{':
            new_tokens += ['OB_', 'NEW_LINE']
        elif token == '*/':
            new_tokens += ['*/', 'NEW_LINE']
        elif token == ';':
            new_tokens += [';', 'NEW_LINE']
        else:
            new_tokens.append(token)

        if i < len(tokens_and_types) - 1 and tokens_and_types[i + 1][0] in TOK_NO_SPACE_BEFORE:
            next_token = tokens_and_types[i + 1][0]
            new_tokens[len(new_tokens) - 1] += next_token
            if next_token == ';':
                new_tokens.append('NEW_LINE')
            i += 2
            continue
        i += 1

    lines = re.split('NEW_LINE', ' '.join(new_tokens))

    untok_s = indent_lines(lines)
    untok_s = untok_s.replace('CB_COLON', '};').replace(
        'CB_COMA', '},').replace('CB_', '}').replace('OB_', '{')
    return untok_s


def indent_lines(lines):
    prefix = ''
    for i, line in enumerate(lines):
        line = line.strip()
        if re.match('CB_COLON|CB_COMA|CB_', line):
            prefix = prefix[2:]
            line = prefix + line
        elif line.endswith('OB_'):
            line = prefix + line
            prefix += '  '
        else:
            line = prefix + line
        lines[i] = line
    untok_s = '\n'.join(lines)
    return untok_s


def detokenize_java(s):
    assert isinstance(s, str) or isinstance(s, list)
    if isinstance(s, list):
        s = ' '.join(s)
    s = s.replace('ENDCOM', 'NEW_LINE')
    s = s.replace('▁', 'SPACETOKEN')

    s = s.replace('} "', 'CB_ "')
    s = s.replace('" {', '" OB_')
    s = s.replace('*/ ', '*/ NEW_LINE')
    s = s.replace('} ;', 'CB_COLON NEW_LINE')
    s = s.replace('} ,', 'CB_COMA')
    s = s.replace('}', 'CB_ NEW_LINE')
    s = s.replace('{', 'OB_ NEW_LINE')
    s = s.replace(';', '; NEW_LINE')
    lines = re.split('NEW_LINE', s)

    untok_s = indent_lines(lines)
    untok_s = untok_s.replace('CB_COLON', '};').replace(
        'CB_COMA', '},').replace('CB_', '}').replace('OB_', '{')
    untok_s = untok_s.replace('> > >', '>>>').replace('<< <', '<<<')
    untok_s = untok_s.replace('> >', '>>').replace('< <', '<<')

    try:
        # call parser of the tokenizer to find comments and string and detokenize them correctly
        tokens_generator = javalang_tok.tokenize(untok_s, keep_comments=True)
        for token in tokens_generator:
            if isinstance(token, javalang_tok.String) or isinstance(token, javalang_tok.Comment):
                token_ = token.value.replace('STRNEWLINE', '\n').replace('TABSYMBOL', '\t').replace(' ', '').replace(
                    'SPACETOKEN', ' ')
                untok_s = untok_s.replace(token.value, token_)
    except KeyboardInterrupt:
        raise
    except:
        pass
    return untok_s


def extract_functions_java(s):
    tokens = s.split()
    i = ind_iter(len(tokens))
    functions_standalone = []
    functions_class = []
    try:
        token = tokens[i.i]
    except KeyboardInterrupt:
        raise
    except:
        return [], []
    while True:
        try:
            # detect function
            if token == ')' and (tokens[i.i + 1] == '{' or (tokens[i.i + 1] == 'throws' and tokens[i.i + 3] == '{')):
                # go previous until the start of function
                while token not in [';', '}', '{', '*/', 'ENDCOM']:
                    i.prev()
                    token = tokens[i.i]

                if token == '*/':
                    while token != '/*':
                        i.prev()
                        token = tokens[i.i]
                    function = [token]
                    while token != '*/':
                        i.next()
                        token = tokens[i.i]
                        function.append(token)
                elif token == 'ENDCOM':
                    while token != '//':
                        i.prev()
                        token = tokens[i.i]
                    function = [token]
                    while token != 'ENDCOM':
                        i.next()
                        token = tokens[i.i]
                        function.append(token)
                else:
                    i.next()
                    token = tokens[i.i]
                    function = [token]

                while token != '{':
                    i.next()
                    token = tokens[i.i]
                    function.append(token)
                if token == '{':
                    number_indent = 1
                    while not (token == '}' and number_indent == 0):
                        try:
                            i.next()
                            token = tokens[i.i]
                            if token == '{':
                                number_indent += 1
                            elif token == '}':
                                number_indent -= 1
                            function.append(token)
                        except StopIteration:
                            break
                    if 'static' in function[0:function.index('{')]:
                        functions_standalone.append(
                            remove_java_annotation(' '.join(function)))
                    else:
                        functions_class.append(
                            remove_java_annotation(' '.join(function)))
            i.next()
            token = tokens[i.i]
        except KeyboardInterrupt:
            raise
        except:
            break
    return functions_standalone, functions_class


def extract_functions_java_with_docstring(function):
    ds = re.findall('[/][*].*?[*][/][ ]', function, re.DOTALL)
    if len(ds) > 0:
        for d in ds:
            function = function.replace(d, '')
        coms = ' '.join([d[:-1] for d in ds])
        inline_coms = re.findall('[/][/].*?[E][N][D][C][O][M]', function)
        for inline_com in inline_coms:
            function = function.replace(inline_com, '')
            coms += ' <INLINE> '
            coms += inline_com
        if len(re.sub(r'\W', '', coms.replace('<INLINE>', '').replace('ENDCOM', ''))) < 5:
            return '', ''
        else:
            return re.sub('\s+', ' ', function), coms
    else:
        return '', ''


def clean_hashtags_functions_cpp(function):
    function = re.sub('[#][ ][i][n][c][l][u][d][e][ ]["].*?["]', "", function)
    function = re.sub('[#][ ][i][n][c][l][u][d][e][ ][<].*?[>]', "", function)
    function = re.sub('[#][ ][i][f][n][d][e][f][ ][^ ]*', "", function)
    function = re.sub('[#][ ][i][f][d][e][f][ ][^ ]*', "", function)
    function = re.sub(
        '[#][ ][d][e][f][i][n][e][ ][^ ]*[ ][(][ ].*?[ ][)][ ][(][ ].*[ ][)]', "", function)
    function = re.sub(
        '[#][ ][d][e][f][i][n][e][ ][^ ]*[ ][(][ ].*?[ ][)][ ][{][ ].*[ ][}]', "", function)
    function = re.sub(
        '[#][ ][d][e][f][i][n][e][ ][^ ]*[ ]([(][ ])?["].*?["]([ ][)])?', "", function)
    function = re.sub(
        '[#][ ][d][e][f][i][n][e][ ][^ ]*[ ]([(][ ])?\d*\.?\d*([ ][+-/*][ ]?\d*\.?\d*)?([ ][)])?', "", function)
    function = re.sub('[#][ ][d][e][f][i][n][e][ ][^ ]', "", function)
    function = re.sub(
        '[#][ ][i][f][ ][d][e][f][i][n][e][d][ ][(][ ].*?[ ][)]', "", function)
    function = re.sub('[#][ ][i][f][ ][^ ]*', "", function)
    function = function.replace('# else', '')
    function = function.replace('# endif', '')
    function = function.strip()
    return function


def extract_functions_cpp(s):
    try:
        s = clean_hashtags_functions_cpp(s)
        s = s.replace('ENDCOM', '\n').replace('▁', 'SPACETOKEN')
        tokens = get_cpp_tokens_and_types(s)
    except:
        return [], []
    i = ind_iter(len(tokens))
    functions_standalone = []
    functions_class = []
    try:
        token, token_type = tokens[i.i]
    except:
        return [], []
    while True:
        try:
            # detect function
            if token == ')' and ((tokens[i.i + 1][0] == '{' and tokens[i.i + 2][0] != '}') or (
                    tokens[i.i + 1][0] == 'throw' and tokens[i.i + 4][0] == '{' and tokens[i.i + 5][0] == '}')):
                # go previous until the start of function
                while token not in {';', '}', '{'}:
                    try:
                        i.prev()
                    except StopIteration:
                        break
                    token = tokens[i.i][0]

                i.next()
                token, token_type = tokens[i.i]
                if token_type == TokenKind.COMMENT:
                    token = token.strip()
                    token += " ENDCOM"
                function = [token]
                token_types = [token_type]
                while token != '{':
                    i.next()
                    token, token_type = tokens[i.i]
                    if token_type == TokenKind.COMMENT:
                        token = token.strip()
                        token += " ENDCOM"
                    function.append(token)
                    token_types.append(token_type)

                if token_types[function.index('(') - 1] != TokenKind.IDENTIFIER:
                    continue
                if token == '{':
                    number_indent = 1
                    while not (token == '}' and number_indent == 0):
                        try:
                            i.next()
                            token, token_type = tokens[i.i]
                            if token == '{':
                                number_indent += 1
                            elif token == '}':
                                number_indent -= 1
                            if token_type == TokenKind.COMMENT:
                                token = token.strip()
                                token += " ENDCOM"
                            function.append(token)
                        except StopIteration:
                            break
                    if 'static' in function[0:function.index('{')] or '::' not in function[0:function.index('(')]:
                        function = ' '.join(function)
                        function = re.sub(
                            "[<][ ][D][O][C][U][M][E][N][T].*?[>] ", "", function)
                        function = clean_hashtags_functions_cpp(function)
                        function = function.strip()
                        function = function.replace(
                            '\n', 'ENDCOM').replace('SPACETOKEN', '▁')
                        if not re.sub('[^ ]*[ ][(][ ]\w*([ ][,][ ]\w*)*[ ][)]', "", function[:function.index('{')]).strip().startswith('{') and not function.startswith('#'):
                            functions_standalone.append(function)
                    else:
                        function = ' '.join(function)
                        function = re.sub(
                            "[<][ ][D][O][C][U][M][E][N][T].*?[>] ", "", function)
                        function = clean_hashtags_functions_cpp(function)
                        function = function.strip()
                        function = function.replace(
                            '\n', 'ENDCOM').replace('SPACETOKEN', '▁')
                        if not re.sub('[^ ]*[ ][(][ ]\w*([ ][,][ ]\w*)*[ ][)]', "", function[:function.index('{')]).strip().startswith('{') and not function.startswith('#'):
                            functions_class.append(function)
            i.next()
            token = tokens[i.i][0]
        except:
            break
    return functions_standalone, functions_class


def extract_functions_cpp_with_docstring(function):
    function = re.sub("[<][ ][D][O][C][U][M][E][N][T].*?[>] ", "", function)
    ds = re.findall('[/][*].*?[*][/][ ]', function, re.DOTALL)
    if len(ds) > 0:
        for d in ds:
            function = function.replace(d, '')
        coms = ' '.join([d[:-1] for d in ds])
        inline_coms = re.findall('[/][/].*?[E][N][D][C][O][M]', function)
        for inline_com in inline_coms:
            function = function.replace(inline_com, '')
            coms += ' <INLINE> '
            coms += inline_com
        if len(re.sub(r'\W', '', coms.replace('<INLINE>', '').replace('ENDCOM', ''))) < 5:
            return '', ''
        else:
            return re.sub('\s+', ' ', function), coms
    else:
        return '', ''


def remove_java_annotation(function):
    return re.sub('^(@ (Override|Deprecated|SuppressWarnings) (\( .* \) )?)*', '', function)


def get_first_token_before_first_parenthesis(s):
    assert isinstance(s, str) or isinstance(s, list)
    if isinstance(s, str):
        s = s.split()
    return s[s.index('(') - 1]


def get_function_name_java(s):
    return get_first_token_before_first_parenthesis(s)


def get_function_name_cpp(s):
    return get_first_token_before_first_parenthesis(s)


def extract_arguments_java(f):
    return extract_arguments_java_using_parentheses(f)


def extract_arguments_cpp(f):
    return extract_arguments_java_using_parentheses(f)


def extract_arguments_java_using_parentheses(f):
    f = f.split(' ')
    types = []
    names = []
    par = 0
    arguments = []
    f = f[f.index('('):]
    for tok in f:
        if tok == '(':
            par += 1
        elif tok == ')':
            par -= 1
        arguments.append(tok)
        if par == 0:
            break
    arguments = ' '.join(arguments[1:-1])
    if arguments == '':
        return ['None'], ['None']
    arguments = arguments.split(',')
    for arg in arguments:
        bracks = re.findall('\[ \]', arg)
        bracks = ' '.join(bracks)
        arg = arg.replace(bracks, '')
        arg = arg.strip()
        arg = re.sub(' +', ' ', arg)
        t = ' '.join(arg.split(' ')[:-1] + [bracks])
        n = arg.split(' ')[-1]
        types.append(t)
        names.append(n)
    return types, names


In [ ]:
import os
import numpy as np
import torch
from logging import getLogger


logger = getLogger()


BOS_WORD = '<s>'
EOS_WORD = '</s>'
PAD_WORD = '<pad>'
UNK_WORD = '<unk>'

SPECIAL_WORD = '<special%i>'
SPECIAL_WORDS = 10

SEP_WORD = SPECIAL_WORD % 0
MASK_WORD = SPECIAL_WORD % 1


class Dictionary(object):

    def __init__(self, id2word, word2id, counts):
        assert len(id2word) == len(word2id) == len(counts)
        self.id2word = id2word
        self.word2id = word2id
        self.counts = counts
        self.bos_index = word2id[BOS_WORD]
        self.eos_index = word2id[EOS_WORD]
        self.pad_index = word2id[PAD_WORD]
        self.unk_index = word2id[UNK_WORD]
        self.check_valid()

    def __len__(self):
        """
        Returns the number of words in the dictionary.
        """
        return len(self.id2word)

    def __getitem__(self, i):
        """
        Returns the word of the specified index.
        """
        return self.id2word[i]

    def __contains__(self, w):
        """
        Returns whether a word is in the dictionary.
        """
        return w in self.word2id

    def __eq__(self, y):
        """
        Compare this dictionary with another one.
        """
        self.check_valid()
        y.check_valid()
        if len(self.id2word) != len(y):
            return False
        return all(self.id2word[i] == y[i] for i in range(len(y)))

    def check_valid(self):
        """
        Check that the dictionary is valid.
        """
        assert self.bos_index == 0
        assert self.eos_index == 1
        assert self.pad_index == 2
        assert self.unk_index == 3
        assert all(self.id2word[4 + i] == SPECIAL_WORD %
                   i for i in range(SPECIAL_WORDS))
        assert len(self.id2word) == len(self.word2id) == len(self.counts)
        assert set(self.word2id.keys()) == set(self.counts.keys())
        for i in range(len(self.id2word)):
            assert self.word2id[self.id2word[i]] == i
        last_count = 1e18
        for i in range(4 + SPECIAL_WORDS, len(self.id2word) - 1):
            count = self.counts[self.id2word[i]]
            assert count <= last_count
            last_count = count

    def index(self, word, no_unk=False):
        """
        Returns the index of the specified word.
        """
        if no_unk:
            return self.word2id[word]
        else:
            return self.word2id.get(word, self.unk_index)

    def max_vocab(self, max_vocab):
        """
        Limit the vocabulary size.
        """
        assert max_vocab >= 1
        init_size = len(self)
        self.id2word = {k: v for k, v in self.id2word.items() if k < max_vocab}
        self.word2id = {v: k for k, v in self.id2word.items()}
        self.counts = {k: v for k, v in self.counts.items()
                       if k in self.word2id}
        self.check_valid()
        logger.info("Maximum vocabulary size: %i. Dictionary size: %i -> %i (removed %i words)."
                    % (max_vocab, init_size, len(self), init_size - len(self)))

    def min_count(self, min_count):
        """
        Threshold on the word frequency counts.
        """
        assert min_count >= 0
        init_size = len(self)
        self.id2word = {k: v for k, v in self.id2word.items(
        ) if self.counts[self.id2word[k]] >= min_count or k < 4 + SPECIAL_WORDS}
        self.word2id = {v: k for k, v in self.id2word.items()}
        self.counts = {k: v for k, v in self.counts.items()
                       if k in self.word2id}
        self.check_valid()
        logger.info("Minimum frequency count: %i. Dictionary size: %i -> %i (removed %i words)."
                    % (min_count, init_size, len(self), init_size - len(self)))

    @staticmethod
    def read_vocab(vocab_path):
        """
        Create a dictionary from a vocabulary file.
        """
        skipped = 0
        assert os.path.isfile(vocab_path), vocab_path
        word2id = {BOS_WORD: 0, EOS_WORD: 1, PAD_WORD: 2, UNK_WORD: 3}
        for i in range(SPECIAL_WORDS):
            word2id[SPECIAL_WORD % i] = 4 + i
        counts = {k: 0 for k in word2id.keys()}
        f = open(vocab_path, 'r', encoding='utf-8')
        for i, line in enumerate(f):
            if '\u2028' in line:
                skipped += 1
                continue
            line = line.rstrip().split()
            if len(line) != 2:
                skipped += 1
                continue
            assert len(line) == 2, (i, line)
            # assert line[0] not in word2id and line[1].isdigit(), (i, line)
            assert line[1].isdigit(), (i, line)
            if line[0] in word2id:
                skipped += 1
                print('%s already in vocab' % line[0])
                continue
            if not line[1].isdigit():
                skipped += 1
                print('Empty word at line %s with count %s' % (i, line))
                continue
            word2id[line[0]] = 4 + SPECIAL_WORDS + i - \
                skipped  # shift because of extra words
            counts[line[0]] = int(line[1])
        f.close()
        id2word = {v: k for k, v in word2id.items()}
        dico = Dictionary(id2word, word2id, counts)
        logger.info("Read %i words from the vocabulary file." % len(dico))
        if skipped > 0:
            logger.warning("Skipped %i empty lines!" % skipped)
        return dico

    @staticmethod
    def index_data(path, bin_path, dico):
        """
        Index sentences with a dictionary.
        """
        if bin_path is not None and os.path.isfile(bin_path):
            print("Loading data from %s ..." % bin_path)
            data = torch.load(bin_path)
            assert dico == data['dico']
            return data

        positions = []
        sentences = []
        unk_words = {}

        # index sentences
        f = open(path, 'r', encoding='utf-8')
        for i, line in enumerate(f):
            if i % 1000000 == 0 and i > 0:
                print(i)
            s = line.rstrip().split()
            # skip empty sentences
            if len(s) == 0:
                print("Empty sentence in line %i." % i)
            # index sentence words
            count_unk = 0
            indexed = []
            for w in s:
                word_id = dico.index(w, no_unk=False)
                # if we find a special word which is not an unknown word, skip the sentence
                if 0 <= word_id < 4 + SPECIAL_WORDS and word_id != 3:
                    logger.warning(
                        'Found unexpected special word "%s" (%i)!!' % (w, word_id))
                    continue
                assert word_id >= 0
                indexed.append(word_id)
                if word_id == dico.unk_index:
                    unk_words[w] = unk_words.get(w, 0) + 1
                    count_unk += 1
            # add sentence
            positions.append([len(sentences), len(sentences) + len(indexed)])
            sentences.extend(indexed)
            sentences.append(1)  # EOS index
        f.close()

        # tensorize data
        positions = np.int64(positions)
        if len(dico) < 1 << 16:
            sentences = np.uint16(sentences)
        elif len(dico) < 1 << 31:
            sentences = np.int32(sentences)
        else:
            raise Exception("Dictionary is too big.")
        assert sentences.min() >= 0
        data = {
            'dico': dico,
            'positions': positions,
            'sentences': sentences,
            'unk_words': unk_words,
        }
        if bin_path is not None:
            print("Saving the data to %s ..." % bin_path)
            torch.save(data, bin_path, pickle_protocol=4)

        return data


In [ ]:
### Generating Code Vocab files with fastBPE 

In [ ]:
!cp /content/drive/MyDrive/EVA4/END_Capstone/TransCoder/* .

In [ ]:
nl_pl_df = pd.read_csv("/content/end_capstone.csv")
# nl_pl_df['docstring'].to_csv("/content/end_capstone_docstring.txt",index=False)
# nl_pl_df['code'].to_csv("/content/end_capstone_code.txt",index=False)
with open("enc_code_full.txt",'w') as f:
    for code_piece in nl_pl_df.loc[:, 'code']:
        toks = tokenize_python(code_piece, keep_comments=False)
        f.write(' '.join(toks))
        f.write('\n')

In [ ]:
%%bash
./fast learnbpe 10000 /content/enc_code_full.txt > python_codes_full.txt
./fast getvocab /content/enc_code_full.txt > vocab_full.txt
./fast applybpe python_code_applybpe_full.txt /content/enc_code_full.txt python_codes_full.txt

Loading vocabulary from /content/enc_code_full.txt ...
Read 276934 words (5686 unique) from text file.
tcmalloc: large alloc 12000002048 bytes == 0x561cca8c2000 @  0x7fe73d35d887 0x561cc81548f3 0x561cc814978f 0x7fe73c798bf7 0x561cc8149a1a
Loading vocabulary from /content/enc_code_full.txt ...
Read 276934 words (5686 unique) from text file.
Loading codes from python_codes_full.txt ...
Read 10000 codes from the codes file.
Loading vocabulary from /content/enc_code_full.txt ...
Read 276934 words (5686 unique) from text file.
Applying BPE to /content/enc_code_full.txt ...
Modified 276934 words from text file.


In [ ]:
### Creating the vocab
my_dict = Dictionary.read_vocab("/content/vocab_full.txt")

In [ ]:
complete_data = Dictionary.index_data('/content/enc_code_full.txt', '/content/enc_code_full.txt.pth', my_dict)

Loading data from /content/enc_code_full.txt.pth ...


In [ ]:
for index in data['positions'][:10]:
    print(index[0],index[1])

In [ ]:
list(data['sentences'][0:27])

In [ ]:
for index in data['positions'][:10]:
    start,end = index[0], index[1]
    [self.code_tok_vectorizer.bos_index] + tok_ids + [self.code_tok_vectorizer.eos_index]
    #print(' '.join([ my_dict.id2word[idx] for idx in data['sentences'][start:end]]))

In [ ]:
my_dict.pad_index

2

### New NLP DS

In [ ]:
tok_ids_list=[]
class NLPLSingleEntry(object):
    """A single set of features of data."""

    def __init__(self, 
                 code_ids,
                 code_mask, 
                 doc_ids,
                 doc_mask,
                 ):
        self.code_ids = code_ids
        self.code_mask = code_mask
        #self.tok_ids = tok_ids
        self.doc_ids = doc_ids
        self.doc_mask = doc_mask
        #self.segment_ids = segment_ids
        #self.label_id = label_id
class NLPLDataSet():
    def __init__(self, 
                 doc_tokenizer, 
                 code_tokenizer,
                 code_tok_vectorizer):
        self.doc_tokenizer = doc_tokenizer
        self.code_tokenizer = code_tokenizer
        self.code_tok_vectorizer = code_tok_vectorizer

    def prepare_tokens(self, 
                       samples, 
                       tokenizer, 
                       max_seq_length=0,
                       data_type=None):
        """
            Tokenizes an input sequence, adds padding and SOS+EOS 
        """
        toks = tokenizer.tokenize(samples.lower())
        # print(data_type)
        # print(toks)
        if max_seq_length > 2 and len(toks) > max_seq_length - 2:
            toks = toks[:max_seq_length -2]
        tok_ids =  tokenizer.convert_tokens_to_ids(toks)
        ### We use pseudo-BERT process so we will add both CLS and SEP tokens for
        ### src and target inputs
        tok_ids = [tokenizer.cls_token_id] + tok_ids + [tokenizer.sep_token_id]
        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [ 1 ] * len(tok_ids)

        if len(tok_ids) < max_seq_length:
            padding_length = max_seq_length - len(tok_ids)
            tok_ids = tok_ids + ([tokenizer.pad_token_id] * padding_length)
            input_mask = input_mask + ([ 0 ] * padding_length) ### Padded tokens are zero-masked
        
        # print(tok_ids)
        return tok_ids, input_mask
    def prepare_code_tokens(self, 
                       samples, 
                       tokenizer, 
                       max_seq_length=0,
                       data_type=None):
        """
            Tokenizes an input sequence, adds padding and SOS+EOS 
        """
        #_toks, _tok_types = get_lang_specific_tokens(self.code_tokenizer, samples)
        tok_ids = samples
        #print(_tok_types)
        # print(data_type)
        # print(toks)
        if max_seq_length > 2 and len(tok_ids) > max_seq_length - 2:
            tok_ids = tok_ids[:max_seq_length -2]
            #_tok_types = _tok_types[:max_seq_length -2]
        #tok_ids =  tokenizer.convert_tokens_to_ids(toks)
        #tok_ids = [ self.code_tok_vectorizer.convert_codepiece_to_id(code) for code in _toks]
        #tok_types = [ self.code_tok_vectorizer.convert_toktype_to_id(toktype) for toktype in _tok_types]
        
        ### We use pseudo-BERT process so we will add both CLS and SEP tokens for
        ### src and target inputs
        tok_ids = [self.code_tok_vectorizer.bos_index] + tok_ids + [self.code_tok_vectorizer.eos_index]
        #tok_types = [self.code_tok_vectorizer.ID_SOS_FOR_TOKEN_TYPE] + tok_types + [self.code_tok_vectorizer.ID_EOS_FOR_TOKEN_TYPE]
        #print(len(tok_ids), len(tok_types))
        #assert(len(tok_ids) == len(tok_types))
        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [ 1 ] * len(tok_ids)

        if len(tok_ids) < max_seq_length:
            padding_length = max_seq_length - len(tok_ids)
            tok_ids = tok_ids + ([self.code_tok_vectorizer.pad_index] * padding_length)
            #tok_types = tok_types + ([self.code_tok_vectorizer.ID_PAD_FOR_TOKEN_TYPE] * padding_length)
            input_mask = input_mask + ([ 0 ] * padding_length) ### Padded tokens are zero-masked
        
        # print(tok_ids)
        #return tok_ids, tok_types, input_mask
        return tok_ids, input_mask

    def create_dataset(self,
                    nl_to_pl_df,
                    code_data_array,
                    final_ds,
                    sample_count=10000,
                    max_doc_len=50,
                    max_code_len=0):
        """
            Reads from a dataframe, tokenizes and numericalizes both docstrings 
            and code. 

        """
        #final_ds = []
        for idx in nl_to_pl_df.itertuples():
            ## For SOS and EOS tokens 2 positions are left
            if not idx.cleaned_code:
                print("Invalid entry, No code found for:", idx.docstring)
            
            doc_toks, doc_mask = self.prepare_tokens(idx.docstring,
                                                      self.doc_tokenizer,
                                                      max_doc_len,
                                                      "docs")
        for index in code_data_array['positions']:
            start,end = index[0], index[1]
            code_array = list(code_data_array['sentences'][start:end])
            code_ids, code_mask = self.prepare_code_tokens(code_array,
                                                            None,
                                                            max_code_len,
                                                            "code")
            #code_toks = None
            ### Skip over current iteration if no valid code found

            # print(code_toks)
            # print(code_mask)
            # print(doc_toks)
            # print(doc_mask)
            final_entry = NLPLSingleEntry(code_ids,
                                          code_mask, 
                                          doc_toks, 
                                          doc_mask)
            # print(final_entry.code_ids)
            # print(final_entry.code_mask)
            # print(final_entry.doc_ids)
            # print(final_entry.doc_mask)
            final_ds.append(final_entry)
        #print(len(final_ds))
        return final_ds
        

In [ ]:
### We will use the same tokenizer for both docstrings and code
final_ds = []
MAX_LENGTH=512
#selected_elems = nl_to_pl_df[nl_to_pl_df['cleaned_code_len'] <= MAX_LENGTH]
#selected_elems = my_df_copy[my_df_copy['code_len'] <= MAX_LENGTH]
MAX_VOCAB_LENGTH=128
assert(MAX_VOCAB_LENGTH <= MAX_LENGTH)
#selected_elems = nl_to_pl_df[nl_to_pl_df['cleaned_code_len'] <= MAX_LENGTH]
#selected_elems = my_df_copy[my_df_copy['code_len'] <= MAX_LENGTH]
#init_tokenizer=PythonTokenizer2()
selected_elems = nl_to_pl_df[nl_to_pl_df['cleaned_code_len'] <= MAX_VOCAB_LENGTH]
selected_elems_file_path = '/content/enc_code_local.txt'
selected_elems_saved_dict = selected_elems_file_path+'.pth'
my_dict = Dictionary.read_vocab("/content/vocab_full.txt")

with open(selected_elems_file_path,'w') as f:
    for code_piece in selected_elems.loc[:, 'code']:
        toks = tokenize_python(code_piece, keep_comments=False)
        f.write(' '.join(toks))
        f.write('\n')
complete_data = Dictionary.index_data(selected_elems_file_path, 
                                      selected_elems_saved_dict, 
                                      my_dict)

my_nlpl_ds = NLPLDataSet(auto_tokenizer, None, my_dict).create_dataset(selected_elems,
                                                                        complete_data,
                                                                        final_ds, 
                                                                        max_doc_len=MAX_VOCAB_LENGTH, 
                                                                        max_code_len=MAX_VOCAB_LENGTH)


Saving the data to /content/enc_code_local.txt.pth ...


In [ ]:
my_nlpl_ds

In [ ]:
all_code_ids = torch.tensor([f.code_ids for f in my_nlpl_ds], dtype=torch.long)
all_code_mask = torch.tensor([f.code_mask for f in my_nlpl_ds], dtype=torch.long)
all_doc_ids = torch.tensor([f.doc_ids for f in my_nlpl_ds], dtype=torch.long)
all_doc_mask = torch.tensor([f.doc_mask for f in my_nlpl_ds], dtype=torch.long)
#all_tok_ids = torch.tensor([f.tok_ids for f in my_nlpl_ds], dtype=torch.long)
# if output_mode == "classification":
#     all_label_ids = torch.tensor([f.label_id for f in my_nlpl_ds], dtype=torch.long)

train_dataset = TensorDataset(all_code_ids,
                              all_code_mask, 
                              all_doc_ids, 
                              all_doc_mask)
 #                             all_tok_ids)




In [ ]:
len(my_dict.id2word.keys())

5700

In [ ]:
dataset_size = len(train_dataset)
dataset_indices = list(range(dataset_size))
np.random.shuffle(dataset_indices)
val_split_index = int(np.floor(0.2 * dataset_size))

train_idx, val_idx = dataset_indices[val_split_index:], dataset_indices[:val_split_index]

In [ ]:
BATCH_SIZE=32
train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(val_idx)


#train_sampler = RandomSampler(train_dataset,) #if args.local_rank == -1 else DistributedSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE, shuffle=False)
val_dataloader = DataLoader(train_dataset, sampler=val_sampler, batch_size=BATCH_SIZE, shuffle=False)

#train_sampler = RandomSampler(train_dataset) #if args.local_rank == -1 else DistributedSampler(train_dataset)

train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)

In [ ]:
iter_one = next(iter(train_dataloader))
iter_one_val = next(iter(val_dataloader))

#iter_one[0]=all_code_ids, 
#iter_one[1]=all_code_mask, 
#iter_one[2]=all_doc_ids, 
#iter_one[3]=all_doc_mask
#mask_reshape.shape
# trg = iter_one[0]
# trg_len = trg.shape[1]
# trg_pad_mask = iter_one[1].unsqueeze(1).unsqueeze(2) 
# trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len))).bool()

# #trg_sub_mask = [trg len, trg len]
    
# trg_mask = trg_pad_mask & trg_sub_mask

In [ ]:
iter_one[-1].shape

torch.Size([8, 128])

### Model Definitions

In [ ]:
class TransEncoder(nn.Module):
    def __init__(self, 
                 input_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim,
                 dropout, 
                 device,
                 max_length = MAX_LENGTH):
        super().__init__()

        self.device = device
        
        self.tok_embedding = nn.Embedding(input_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([TransEncoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim,
                                                  dropout, 
                                                  device) 
                                     for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len]
        #src_mask = [batch size, 1, 1, src len]
        
        batch_size = src.shape[0]
        src_len = src.shape[1]
        
        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        
        #pos = [batch size, src len]
        
        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))
        
        #src = [batch size, src len, hid dim]
        
        for layer in self.layers:
            src = layer(src, src_mask)
            
        #src = [batch size, src len, hid dim]
            
        return src

In [ ]:
class TransEncoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim,  
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len, hid dim]
        #src_mask = [batch size, 1, 1, src len] 
                
        #self attention
        _src, _ = self.self_attention(src, src, src, src_mask)
        
        #dropout, residual connection and layer norm
        src = self.self_attn_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        #positionwise feedforward
        _src = self.positionwise_feedforward(src)
        
        #dropout, residual and layer norm
        src = self.ff_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        return src

In [ ]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()
        
        assert hid_dim % n_heads == 0
        
        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads
        
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)
        
        self.fc_o = nn.Linear(hid_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
        
    def forward(self, query, key, value, mask = None):
        
        batch_size = query.shape[0]
        
        #query = [batch size, query len, hid dim]
        #key = [batch size, key len, hid dim]
        #value = [batch size, value len, hid dim]
                
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)
        
        #Q = [batch size, query len, hid dim]
        #K = [batch size, key len, hid dim]
        #V = [batch size, value len, hid dim]
                
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        
        #Q = [batch size, n heads, query len, head dim]
        #K = [batch size, n heads, key len, head dim]
        #V = [batch size, n heads, value len, head dim]
                
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale
        
        #energy = [batch size, n heads, query len, key len]
        
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)
        
        attention = torch.softmax(energy, dim = -1)
                
        #attention = [batch size, n heads, query len, key len]
                
        x = torch.matmul(self.dropout(attention), V)
        
        #x = [batch size, n heads, query len, head dim]
        
        x = x.permute(0, 2, 1, 3).contiguous()
        
        #x = [batch size, query len, n heads, head dim]
        
        x = x.view(batch_size, -1, self.hid_dim)
        
        #x = [batch size, query len, hid dim]
        
        x = self.fc_o(x)
        
        #x = [batch size, query len, hid dim]
        
        return x, attention

In [ ]:
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()
        
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [batch size, seq len, hid dim]
        
        x = self.dropout(torch.relu(self.fc_1(x)))
        
        #x = [batch size, seq len, pf dim]
        
        x = self.fc_2(x)
        
        #x = [batch size, seq len, hid dim]
        
        return x

In [ ]:
class TransDecoder(nn.Module):
    def __init__(self, 
                 output_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device,
                 max_length = 1000,
                 tok_type_dim=62):
        super().__init__()
        
        self.device = device
        
        self.tok_embedding = nn.Embedding(output_dim, hid_dim)
        #self.tok_type_embedding = nn.Embedding(tok_type_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([TransDecoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim, 
                                                  dropout, 
                                                  device)
                                     for _ in range(n_layers)])
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        #self.fc_out_tok = nn.Linear(hid_dim, tok_type_dim)
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, trg, enc_src, trg_mask, src_mask, src_tok_types=None):
        
        #trg = [batch size, trg len]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
                
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        
        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
                            
        #pos = [batch size, trg len]
            
        # trg = self.dropout((self.tok_embedding(trg) * self.scale) + 
        #                    self.pos_embedding(pos) + 
        #                    self.tok_type_embedding(src_tok_types))
        trg = self.dropout((self.tok_embedding(trg) * self.scale) + 
                           self.pos_embedding(pos) )
                
        #trg = [batch size, trg len, hid dim]
        
        for layer in self.layers:
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        output = self.fc_out(trg)
        #output_tok = self.fc_out_tok(trg)
        #output =F.softmax(output, dim=2)  
        
        #output = [batch size, trg len, output dim]
            
        #return output, output_tok, attention
        return output, attention

In [ ]:
class TransDecoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.encoder_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len, hid dim]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
        
        #self attention
        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)
        
        #dropout, residual connection and layer norm
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))
            
        #trg = [batch size, trg len, hid dim]
            
        #encoder attention
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)
        # query, key, value
        
        #dropout, residual connection and layer norm
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))
                    
        #trg = [batch size, trg len, hid dim]
        
        #positionwise feedforward
        _trg = self.positionwise_feedforward(trg)
        
        #dropout, residual and layer norm
        trg = self.ff_layer_norm(trg + self.dropout(_trg))
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return trg, attention

In [ ]:
class TransSeq2Seq(nn.Module):
    def __init__(self, 
                 encoder, 
                 decoder, 
                 src_pad_idx, 
                 trg_pad_idx, 
                 device,
                 ):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device
        
    def make_src_mask(self, src_mask):
        
        #src = [batch size, src len]
        
        src_mask = src_mask.unsqueeze(1).unsqueeze(2)

        #src_mask = [batch size, 1, 1, src len]

        return src_mask
    
    def make_trg_mask(self, trg, trg_mask):
        
        #trg = [batch size, trg len]
        trg_pad_mask = trg_mask.unsqueeze(1).unsqueeze(2) 
        """
            A boolean tensor of shape [batch size, 1, 1, trg len]
        """
        
        
        #trg_pad_mask = [batch size, 1, 1, trg len]
        
        trg_len = trg.shape[1]
        
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()
        
        #trg_sub_mask = [trg len, trg len]
            
        trg_mask = trg_pad_mask & trg_sub_mask
        
        #trg_mask = [batch size, 1, trg len, trg len]
        
        return trg_mask

    def forward(self, src, src_mask, trg, trg_mask):
        
        #src = [batch size, src len]
        #trg = [batch size, trg len]
                
        src_mask = self.make_src_mask(src_mask)
        trg_mask = self.make_trg_mask(trg, trg_mask)
        
        #src_mask = [batch size, 1, 1, src len]
        #trg_mask = [batch size, 1, trg len, trg len]
        
        enc_src = self.encoder(src, src_mask)
        
        #enc_src = [batch size, src len, hid dim]
                
        output, attention = self.decoder(trg, 
                                         enc_src, 
                                         trg_mask, 
                                         src_mask, 
                                         None)
        
        #output = [batch size, trg len, output dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return output, attention

In [ ]:
len(my_dict.id2word.keys())

5700

In [ ]:
INPUT_DIM = auto_tokenizer.vocab_size
OUTPUT_DIM = len(my_dict.id2word.keys())
TOK_TYPE_OUTPUT_DIM = 62#code_tok_vectorizer.max_tok_length
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 16
DEC_HEADS = 16
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = TransEncoder(INPUT_DIM, 
              HID_DIM, 
              ENC_LAYERS, 
              ENC_HEADS, 
              ENC_PF_DIM, 
              ENC_DROPOUT, 
              device,
              max_length=MAX_LENGTH)

dec = TransDecoder(OUTPUT_DIM, 
              HID_DIM, 
              DEC_LAYERS, 
              DEC_HEADS, 
              DEC_PF_DIM, 
              DEC_DROPOUT, 
              device,
              max_length=MAX_LENGTH,
              tok_type_dim=TOK_TYPE_OUTPUT_DIM)

In [ ]:
code_tok_vectorizer.max_tok_length

12

In [ ]:
my_dict.pad_index

2

In [ ]:
!nvidia-smi

In [ ]:
!cp /content/drive/MyDrive/EVA4/END_Capstone/end_capstone_baseline_128.pt .

In [ ]:
SRC_PAD_IDX = auto_tokenizer.pad_token_id #SRC.vocab.stoi[SRC.pad_token]
TRG_PAD_IDX = my_dict.pad_index#code_tok_vectorizer.ID_PAD_FOR_TOKEN_TYPE #TRG.vocab.stoi[TRG.pad_token]

model = TransSeq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

In [ ]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)


In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 20,007,748 trainable parameters


In [ ]:
#del model
!nvidia-smi

In [ ]:
model.apply(initialize_weights);
LEARNING_RATE = 0.0005

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [ ]:
iter_one = next(iter(train_dataloader))
batch = tuple(t.to(device) for t in iter_one)
# inputs = {'input_ids': batch[0],
#             'attention_mask': batch[1],
#             'token_type_ids': batch[2] ,
#             # XLM don't use segment_ids
#             'labels': batch[3]}

In [ ]:
def train(model, iterator, optimizer, criterion, clip, device,double_loss=False):
    
    model.train()
    
    epoch_loss = 0
    epoch_crit_loss = 0
    epoch_tok_loss = 0
    
    for i, batch in enumerate(iterator):
        
        trg = batch[0].to(device)
        trg_mask = batch[1].to(device)
        src = batch[2].to(device)
        src_mask = batch[3].to(device)
        #src_tok_type = batch[4].to(device)
        
        optimizer.zero_grad()
        
        output, _ = model(src, src_mask, trg[:,:-1], trg_mask[:,:-1])
        #output, tok_op, _ = model(src, src_mask, trg[:,:-1], trg_mask[:,:-1], src_tok_type[:,:-1])
                
        #output = [batch size, trg len - 1, output dim]
        #trg = [batch size, trg len]
            
        output_dim = output.shape[-1]
            
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:,1:].contiguous().view(-1)
                
        #output = [batch size * trg len - 1, output dim]
        #trg = [batch size * trg len - 1]
        if(double_loss == True):            
            tok_op_output_dim = tok_op.shape[-1]            
            tok_op = tok_op.contiguous().view(-1, tok_op_output_dim)
            src_tok_type = src_tok_type[:,1:].contiguous().view(-1)
            #output = [batch size * trg len - 1, output dim]
            #trg = [batch size * trg len - 1]            
            loss,crit_loss, tok_loss = criterion(output, 
                                                 trg,
                                                 tok_op,
                                                 src_tok_type)
            epoch_crit_loss += crit_loss.item()
            epoch_tok_loss += tok_loss.item()
        else:
            loss = criterion(output, trg)
            
        loss.backward()        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()        
        epoch_loss += loss.item()
    
    if(double_loss == True):  
        print(f'Train\tCrit Loss: {epoch_crit_loss/(len(iterator)):.3f} | Token Loss: {epoch_tok_loss/(len(iterator)):.3f}')

    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion, device,double_loss=False):
    
    model.eval()
    
    epoch_loss = 0
    epoch_crit_loss = 0
    epoch_tok_loss = 0
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            trg = batch[0].to(device)
            trg_mask = batch[1].to(device)
            src = batch[2].to(device)
            src_mask = batch[3].to(device)
            #src_tok_type = batch[4].to(device)

            output, _ = model(src, src_mask, trg[:,:-1], trg_mask[:,:-1])
            #output, tok_op, _ = model(src, src_mask, trg[:,:-1], trg_mask[:,:-1], src_tok_type[:,:-1])
            
            #output = [batch size, trg len - 1, output dim]
            #trg = [batch size, trg len]
            
            output_dim = output.shape[-1]
            
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)
            
            if(double_loss == True):            
                tok_op_output_dim = tok_op.shape[-1]            
                tok_op = tok_op.contiguous().view(-1, tok_op_output_dim)
                src_tok_type = src_tok_type[:,1:].contiguous().view(-1)
                #output = [batch size * trg len - 1, output dim]
                #trg = [batch size * trg len - 1]            
                loss,crit_loss, tok_loss = criterion(output, 
                                                     trg,
                                                     tok_op,
                                                     src_tok_type)
                epoch_crit_loss += crit_loss.item()
                epoch_tok_loss += tok_loss.item()
            else:
                loss = criterion(output, trg)
            

            #output = [batch size * trg len - 1, output dim]
            #trg = [batch size * trg len - 1]
            
            # loss = criterion(output, 
            #                  trg,
            #                  tok_op,
            #                  src_tok_type)

            epoch_loss += loss.item()

    if(double_loss == True):  
        print(f'Val\tCrit Loss: {epoch_crit_loss/(len(iterator)):.3f} | Token Loss: {epoch_tok_loss/(len(iterator)):.3f}')  
    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
TRG_PAD_IDX

2

In [ ]:
# my_torch_weights = torch.ones(auto_tokenizer.vocab_size)
# my_torch_weights[1437] = 2
# criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX, weight=my_torch_weights.to(device) )

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

class WeightedCrossEntropy(nn.Module):
    def __init__(self,
                 code_weights=None,
                 code_ignore_idx=None,
                 tok_type_weights=None,
                 tok_type_ignore_idx=None,
                 mix_ratio=0.5):
        
        super(WeightedCrossEntropy, self).__init__()
        self.code_weights=code_weights
        self.code_ignore_idx=code_ignore_idx
        self.tok_type_weights=tok_type_weights
        self.tok_type_ignore_idx=tok_type_ignore_idx
        self.mix_ratio = mix_ratio
    
    def forward(self, 
                code_output, 
                code_trg,
                tok_type_output,
                tok_type_trg):
        
        # code_criterion = nn.CrossEntropyLoss(ignore_index = self.code_ignore_idx,
        #                                      weight=self.code_weights)
        # toktype_criterion = nn.CrossEntropyLoss(ignore_index = self.tok_type_ignore_idx,
        #                                 weight=self.tok_type_weights)

        code_criterion = F.cross_entropy(code_output, 
                                         code_trg, 
                                         weight=self.code_weights,
                                         ignore_index = self.code_ignore_idx)
        toktype_criterion = F.cross_entropy(tok_type_output, 
                                         tok_type_trg, 
                                         weight=self.tok_type_weights,
                                         ignore_index = self.tok_type_ignore_idx)
        
        total_loss = self.mix_ratio * code_criterion + (1-self.mix_ratio )*toktype_criterion
        return total_loss, code_criterion, toktype_criterion

# criterion = WeightedCrossEntropy(code_ignore_idx=TRG_PAD_IDX, 
#                                  tok_type_ignore_idx=TRG_PAD_IDX,
#                                  mix_ratio=0.9999)

In [ ]:
batch = next(iter(train_dataloader))
trg = batch[0].to(device)
trg_mask = batch[1].to(device)
src = batch[2].to(device)
src_mask = batch[3].to(device)
#src_tok_types = batch[4].to(device)
print(trg.shape, trg_mask.shape, src.shape, src_mask.shape)#, src_tok_types.shape)

torch.Size([16, 128]) torch.Size([16, 128]) torch.Size([16, 128]) torch.Size([16, 128])


In [ ]:
model.train()
#with torch.no_grad():
output,attend_val = model(src, src_mask, trg[:,:-1], trg_mask[:,:-1])#, src_tok_types[:,:-1])
    # output_dim = output.shape[-1]
    
    # output = output.contiguous().view(-1, output_dim)
    # trg = trg[:,1:].contiguous().view(-1)
    
    #output = [batch size * trg len - 1, output dim]
    #trg = [batch size * trg len - 1]
    
    #loss = criterion(output, trg)
    #print(loss.item())

In [ ]:
trg.shape

torch.Size([2032])

In [ ]:
output_dim = output.shape[-1]

output = output.contiguous().view(-1, output_dim)
trg = trg[:,1:].contiguous().view(-1)

#tok_output_dim = tok_op.shape[-1]
#tok_op = tok_op.contiguous().view(-1, tok_output_dim)
#src_tok_types = src_tok_types[:,1:].contiguous().view(-1)
    
#output = [batch size * trg len - 1, output dim]
#trg = [batch size * trg len - 1]

#loss = criterion(output, trg)
#print(loss.item())

In [ ]:
output.shape, trg.shape, tok_op.shape, src_tok_types.shape

In [ ]:
loss = criterion(output, trg)#, tok_op, src_tok_types)

In [ ]:
loss.item()

8.71438217163086

In [ ]:
import time
import math

def run_train_eval_loop(model, 
                        train_dataloader,
                        val_dataloader,
                        optimizer,
                        criterion,
                        device,
                        epochs=20,
                        clip=1,
                        best_valid_loss=float('inf'),
                        file_path='end_capstone_baseline_128.pt',
                        double_loss=False,
                        scheduler=None,
                        mix_ratio=0.5):
    
    criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

    if double_loss == True:    
        criterion = WeightedCrossEntropy(code_ignore_idx=TRG_PAD_IDX, 
                                        tok_type_ignore_idx=TRG_PAD_IDX,
                                        mix_ratio=mix_ratio)            
    
    for epoch in range(epochs):
    
        start_time = time.time()
        
        train_loss = train(model, train_dataloader, optimizer, criterion, clip, device,double_loss=double_loss)
        valid_loss = evaluate(model, val_dataloader, criterion, device,double_loss=double_loss)
        
        if(scheduler is not None):
            scheduler.step(valid_loss)
        end_time = time.time()
        
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save({"model":model.state_dict(),
                "optimizer":optimizer.state_dict(),
                "loss":valid_loss,
                },file_path)
        
        print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

In [ ]:
def display_attention(sentence, translation, attention, n_heads = 8, n_rows = 4, n_cols = 2):
    
    assert n_rows * n_cols == n_heads
    
    fig = plt.figure(figsize=(15,25))
    
    for i in range(n_heads):
        
        ax = fig.add_subplot(n_rows, n_cols, i+1)
        
        _attention = attention.squeeze(0)[i].cpu().detach().numpy()

        cax = ax.matshow(_attention, cmap='bone')

        ax.tick_params(labelsize=12)
        ax.set_xticklabels(['']+['<sos>']+[t.lower() for t in sentence]+['<eos>'], 
                           rotation=45)
        ax.set_yticklabels(['']+translation)

        ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
        ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()

In [ ]:
len(train_dataset)

2191

In [ ]:
file_path='end_capstone_bpe3.pt'
LEARNING_RATE = 0.0001

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
run_train_eval_loop(model,
                    train_dataloader,
                    val_dataloader,
                    optimizer,
                    criterion,
                    device,
                    epochs=30,
                    clip=1.4,
                    best_valid_loss=float('inf'),
                    file_path=file_path,
                    double_loss=False)

Epoch: 01 | Time: 0m 6s
	Train Loss: 7.737 | Train PPL: 2292.302
	 Val. Loss: 6.329 |  Val. PPL: 560.559
Epoch: 02 | Time: 0m 6s
	Train Loss: 6.001 | Train PPL: 403.914
	 Val. Loss: 4.994 |  Val. PPL: 147.546
Epoch: 03 | Time: 0m 6s
	Train Loss: 4.928 | Train PPL: 138.056
	 Val. Loss: 4.721 |  Val. PPL: 112.245
Epoch: 04 | Time: 0m 6s
	Train Loss: 4.690 | Train PPL: 108.806
	 Val. Loss: 4.715 |  Val. PPL: 111.562
Epoch: 05 | Time: 0m 6s
	Train Loss: 4.645 | Train PPL: 104.108
	 Val. Loss: 4.628 |  Val. PPL: 102.346
Epoch: 06 | Time: 0m 6s
	Train Loss: 4.550 | Train PPL:  94.600
	 Val. Loss: 4.346 |  Val. PPL:  77.192
Epoch: 07 | Time: 0m 6s
	Train Loss: 4.376 | Train PPL:  79.515
	 Val. Loss: 4.081 |  Val. PPL:  59.206
Epoch: 08 | Time: 0m 6s
	Train Loss: 4.140 | Train PPL:  62.790
	 Val. Loss: 3.838 |  Val. PPL:  46.433
Epoch: 09 | Time: 0m 7s
	Train Loss: 3.928 | Train PPL:  50.801
	 Val. Loss: 3.682 |  Val. PPL:  39.717
Epoch: 10 | Time: 0m 7s
	Train Loss: 3.760 | Train PPL:  42.941

KeyboardInterrupt: ignored

In [ ]:
torch.save({"model":model.state_dict(),
    "optimizer":optimizer.state_dict(),
    "loss":1.360,
    },"/content/end_capstone_self_encode_sizeCor_lowerCase_128_2.pt")

In [ ]:
file_path='/content/end_capstone_bpe2.pt'

chkpt = torch.load(file_path)
print(chkpt['loss'])
model.load_state_dict(chkpt['model'])

2.4893418295042857


<All keys matched successfully>

In [ ]:
model.load_state_dict(chkpt['model'])

<All keys matched successfully>

In [ ]:
LEARNING_RATE = 0.0005
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
optimizer.load_state_dict(chkpt['optimizer'])
scheduler = ReduceLROnPlateau(optimizer, patience=5, min_lr=1e-8,verbose=True)
run_train_eval_loop(model,
                    train_dataloader,
                    val_dataloader,
                    optimizer,
                    criterion,
                    device,
                    epochs=120,
                    clip=1.4,
                    best_valid_loss=chkpt['loss'],
                    file_path=file_path,
                    double_loss=False,
                    scheduler=scheduler,
                    )

In [ ]:
LEARNING_RATE = 0.0005
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
optimizer.load_state_dict(chkpt['optimizer'])
scheduler = ReduceLROnPlateau(optimizer, patience=5, min_lr=1e-9,verbose=True)
run_train_eval_loop(model,
                    train_dataloader,
                    val_dataloader,
                    optimizer,
                    criterion,
                    device,
                    epochs=50,
                    clip=1,
                    best_valid_loss=chkpt['loss'],
                    file_path=file_path,
                    double_loss=True,
                    scheduler=scheduler,
                    mix_ratio=0.9)

In [ ]:
!cp /content/end_capstone_bpe2.pt /content/drive/MyDrive/EVA4/END_Capstone/

In [ ]:
file_path='/content/drive/MyDrive/EVA4/END_Capstone/end_capstone_self_encode_sizeCor_stage2_128wrn_2.pt'
LEARNING_RATE = 0.0005

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
run_train_eval_loop(model,
                    train_dataloader,
                    val_dataloader,
                    optimizer,
                    criterion,
                    device,
                    epochs=20,
                    clip=1.4,
                    best_valid_loss=5.050,
                    file_path=file_path)

In [ ]:
run_train_eval_loop(model,
                    train_dataloader,
                    val_dataloader,
                    optimizer,
                    criterion,
                    device,
                    epochs=20,
                    clip=1,
                    best_valid_loss=chkpt['loss'],
                    file_path=file_path)

In [ ]:
!cp /content/drive/MyDrive/EVA4/END_Capstone/end_capstone_256_mod2.pt /content/

In [ ]:
!cp tut6-model.pt /content/drive/MyDrive/EVA4/END_Capstone/end_capstone_256_mod2.pt

In [ ]:
src_mask = torch.LongTensor(src_mask).unsqueeze(0)#.to(device)
    
src_mask = model.make_src_mask(src_mask)

In [ ]:
!cp /content/end_capstone_codesearch_256_seqLen512.pt /content/drive/MyDrive/EVA4/END_Capstone

In [ ]:
!cp /content/drive/MyDrive/EVA4/END_Capstone/end_capstone_codesearch_256_seqLen512.pt .

In [ ]:
def get_code(sentence,
             tokenizer,
             model, 
             device, 
             max_len = 100):
    
    model.eval()
    dataset_handler = NLPLDataSet(tokenizer, tokenizer)
    src_indexes, src_mask =  dataset_handler.prepare_tokens(sentence, dataset_handler.doc_tokenizer)
    # if isinstance(sentence, str):
    #     nlp = spacy.load('de')
    #     tokens = [token.text.lower() for token in nlp(sentence)]
    # else:
    #     tokens = [token.lower() for token in sentence]
    #dataset_handler.prepare_tokens()

    #tokens = [tokenizer.cls_token] + tokens + [tokenizer.sep_token]
        
    #src_indexes = [tokenizer.convert_tokens_to_ids[token] for token in tokens]

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)
    src_mask = torch.LongTensor(src_mask).unsqueeze(0).to(device)
    
    src_mask = model.make_src_mask(src_mask)
    
    with torch.no_grad():
        enc_src = model.encoder(src_tensor, src_mask)

    trg_indexes = [tokenizer.cls_token_id]
    #trg_mask = [1]

    for i in range(max_len):

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)

        base_mask = torch.LongTensor([1]*(i+1)).unsqueeze(0).to(device)
        trg_mask = model.make_trg_mask(trg_tensor, base_mask)
        
        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
        
        pred_token = output.argmax(2)[:,-1].item()
        
        trg_indexes.append(pred_token)

        if pred_token == tokenizer.sep_token_id:
            break
    
    trg_tokens = [tokenizer.convert_ids_to_tokens(i) for i in trg_indexes]
    
    return trg_tokens[1:], attention

In [ ]:
my_dict.bos_index

0

In [ ]:
def get_code(sentence,
             doc_tokenizer,
             code_tokenizer,
             code_tok_vectorizer,
             model, 
             device, 
             max_len = 100):
    
    model.eval()
    print(sentence)
    dataset_handler = NLPLDataSet(doc_tokenizer, code_tokenizer, code_tok_vectorizer)
    src_indexes, src_mask =  dataset_handler.prepare_tokens(sentence, dataset_handler.doc_tokenizer)
    # if isinstance(sentence, str):
    #     nlp = spacy.load('de')
    #     tokens = [token.text.lower() for token in nlp(sentence)]
    # else:
    #     tokens = [token.lower() for token in sentence]
    #dataset_handler.prepare_tokens()

    #tokens = [tokenizer.cls_token] + tokens + [tokenizer.sep_token]
        
    #src_indexes = [tokenizer.convert_tokens_to_ids[token] for token in tokens]

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)
    src_mask = torch.LongTensor(src_mask).unsqueeze(0).to(device)
    
    src_mask = model.make_src_mask(src_mask)
    
    with torch.no_grad():
        enc_src = model.encoder(src_tensor, src_mask)

    trg_indexes = [code_tok_vectorizer.bos_index]
    #trg_tok_indexes = [code_tok_vectorizer.ID_SOS_FOR_TOKEN_TYPE]
    #trg_mask = [1]

    for i in range(max_len):

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)
        #trg_tok_tensor = torch.LongTensor(trg_tok_indexes).unsqueeze(0).to(device)

        base_mask = torch.LongTensor([1]*(i+1)).unsqueeze(0).to(device)
        trg_mask = model.make_trg_mask(trg_tensor, base_mask)
        
        with torch.no_grad():
            #output, tok_output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask, trg_tok_tensor)
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
        
        pred_token = output.argmax(2)[:,-1].item()        
        trg_indexes.append(pred_token)

        #pred_tok_type = tok_output.argmax(2)[:,-1].item()
        #trg_tok_indexes.append(pred_tok_type)

        if (pred_token == code_tok_vectorizer.eos_index):
         #or pred_token == code_tok_vectorizer.code_word2idx['___EOS___']):
            break

    trg_tokens = [code_tok_vectorizer.id2word[i] for i in trg_indexes]
    #trg_token_types = [code_tok_vectorizer.convert_id_to_toktype(i) for i in trg_tok_indexes]
    
    #return trg_tokens[1:], trg_token_types[1:], attention
    return trg_tokens[1:], attention

In [ ]:
my_dict.id2word

In [ ]:
!cp /content/end_capstone_self_encode_corr1024.pt /content/drive/MyDrive/EVA4/END_Capstone

In [ ]:
input_text = "program to print upper case of a string"
cleaned_string = input_text#.lower().rstrip('\n').lstrip('#')
splitted_text = auto_tokenizer.tokenize(input_text)
mycode, attention_val = get_code(cleaned_string,
                                 auto_tokenizer, 
                                 None,
                                 my_dict,
                                 model, 
                                 device,
                                 max_len=MAX_LENGTH)

print(detokenize_python(mycode))
#output = auto_tokenizer.convert_tokens_to_string(mycode[:-1])
#print(output)

program to print upper case of a string
def def ( n ) :
    return ( n )
</s>


In [ ]:
mycode

In [ ]:
my_dict.e

In [ ]:
import matplotlib.ticker as ticker

#display_attention(splitted_text, mycode, attention_val[:,2,:,:].unsqueeze(1), n_heads=1, n_rows=1, n_cols=1)
n_heads=4
n_rows=2
n_cols=n_heads/n_rows
display_attention(splitted_text, 
                  mycode, 
                  attention_val[:,:8,:,:], 
                  n_heads=n_heads, 
                  n_rows=n_rows, 
                  n_cols=n_cols)

In [ ]:
all_strings = nl_to_pl_df['docstring'].values
for one_string in all_strings[2000:2049]:
    #splitted_text = auto_tokenizer.tokenize(input_text)
    #input_text = "Write a Python function to add two numbers"
    cleaned_string = one_string#.lower().rstrip('\n').lstrip('#')
    #splitted_text = auto_tokenizer.tokenize(one_string)
    mycode, attention_val = get_code(cleaned_string,
                                    auto_tokenizer, 
                                    None,
                                    my_dict,
                                    model, 
                                    device,
                                    max_len=200)
    print(cleaned_string)
    print(detokenize_python(mycode))
    
    #print(init_tokenizer.untokenize(mycode))

In [ ]:
nl_to_pl_df

In [ ]:
all_strings = nl_to_pl_df['docstring'].values
for one_string in all_strings[100:149]:
    cleaned_string = one_string.rstrip('\n').lstrip('#')
    mycode, attention_val = get_code(cleaned_string,auto_tokenizer, model, device, max_len=512)
    print(cleaned_string)
    print(auto_tokenizer.convert_tokens_to_string(mycode[:-1]))


In [ ]:
def even_odd_num(num):
    max = 0
    for num in num:
         if num % 10 == 0:
                 maxnum = num
         return maxnum

In [ ]:
even_odd_num(2)

In [ ]:
torch.save({"model":model.state_dict(),
            "optimizer":optimizer.state_dict(),
            "loss":1.373
            },'end_capstone_baseline_128.pt')

In [ ]:
n = int(input("How many terms? "))

n1 = 0

for i in range(n+1):
    result = 0
    for i in range(n2, n+1):
        result = result + result*n2
    print(result)

In [ ]:
!cp /content/end_capstone_baseline_128.pt /content/drive/MyDrive/EVA4/END_Capstone

In [ ]:
a = [1, 2, 3, 4, 5]
b = [5, 6, 7, 8]
a.update(b)
print(a)

In [ ]:
a = {1, 2, 3}
b = {3, 4, 5, 6}
a = {1, 2, 2, 3, 4}
x = a[i]*b for (a, b) in zip(a, b) )
print(f"{a}")

In [ ]:
def printSubArrays(arr, start, end):
    if end == len(arr):
        return
    elif start > end:
        return printSubArrays(arr, 0, end + 1)
    else:
            print(arr[start:end + 1])